# Ribet-Herbrand experiments

First lets experiment with the density of irregular primes.

Finding irregular primes, and proving FLT for small values of $p$ was an important early use of computers in number theory.

In [1]:
from IPython.display import clear_output
import numpy as np
from matplotlib import pyplot as plt
import collections
%matplotlib inline

def live_plot(data_dict, figsize=(10,7), title=''):
    clear_output(wait=True)
    plt.figure(figsize=figsize)
    for label,data in data_dict.items():
        plt.plot(data, label=label + " " + str(data[-1].n()))
    plt.title(title)
    plt.grid(True)
    plt.xlabel('π')
    plt.legend(loc='center left') # the plot evolves to the right
    plt.show();

In [2]:
data = collections.defaultdict(list)
tot = 0
irreg = 0
for p in prime_range(3,6000):
    tot += 1
    if 0 in bernoulli_mod_p(p):
        irreg += 1
    data['% irregs'].append(irreg/tot)
    live_plot(data)

KeyboardInterrupt: 

If the numerators of the Bernoulli numbers are uniformly random modulo odd primes, then the index
of irregularity should satisfy a Poisson distribution with mean 1/2; as a special
case, noted by Lehmer and Siegel, the density of the irregular primes should
be $1-e^{-1/2}$.

In [3]:
1- data["% irregs"][-1].n()

0.601904761904762

In [4]:
e^(-1/2).n()

0.606530659712633

Question:
----------
Some primes have the property that there are multiple $2\le k\le p - 3$ for which $p|B_{k}$  i.e. there are two distinct bernoulli numbers $B_k$ whose numerator is divisible by $p$, giving a larger class group?
The number of such $p$ is called the index of irregularity,  check experimentally that the density of primes with index 2 matches this probability distribution.

Question:
------------
Does there exist a prime $p$ such that $p|B_{p-3}$ the extreme case of Ribet-Herbrand.

## Finding modular forms

The first irregular prime is 37 as
https://www.lmfdb.org/NumberField/36.0.7710105884424969623139759010953858981831553019262380893.1
has class number 37
and $v_{37}( B_{32})\ge 1$.

See https://oeis.org/A000928

In [5]:
%display latex
K.<zeta37> = CyclotomicField(37)
print(K.class_group(proof=False))

Class group of order 37 with structure C37 of Cyclotomic Field of order 37 and degree 36


But what is the character through which Galois acts?

In [2]:
I = K.class_group(proof=False).gen(); I

Fractional ideal class (149, zeta37 - 73)

In [3]:
G = K.galois_group()
s = G.gen()
s(zeta37)

zeta37^2

So
$$\chi(s) = 2$$
let us now compute $I^\sigma$

In [4]:
I.ideal().gens_two()

(149, zeta37 - 73)

In [5]:
Is = I.parent()([s(t) for t in I.ideal().gens_two()]); Is

Fractional ideal class (149, zeta37 + 64)

In [7]:
for i in range(36): # takes a minute
    if (Is/(I^i)).is_principal():
        print(i)

32


indeed
$$I^\sigma = I^{32} = I^{ \chi(\sigma)^5}$$
so the ideal class of 
$I$ lies in
$$C(\chi^{5}) = C(\chi^{37-32})$$
where $C = \operatorname{Cl}(K)/ \operatorname{Cl}(K)^{37}$
and the cusp form constructed by Ribet is in:

In [8]:
CuspForms(Gamma1(37))

Cuspidal subspace of dimension 40 of Modular Forms space of dimension 75 for Congruence Subgroup Gamma1(37) of weight 2 over Rational Field

In [23]:
def find_ribet(N,k):
    ord = (N-1)/gcd(N-1, k-2)
    D = DirichletGroup(N, zeta_order=ord, base_ring=CyclotomicField(ord))
    for pp in D.base_ring().primes_above(N):
        Rp = pp.residue_field()
        for om in D:
            #print(om.conrey_number())
            #print(list((Rp(om(i)) , Rp(i)^(k-2)) for i in range(N)))
            if all(Rp(om(i)) == i^(k-2) for i in range(N)):
                break
        else:
            continue
        break
    else:
        raise ValueError
    #omk = om^(k-2)
    omk2 = om.minimize_base_ring()
    #for pp2 in D.base_ring().primes_above(N):
    #pp = D.base_ring().ideal(pp)
    print (omk2.order(), omk2.conrey_number())
    S = CuspForms(omk2)
    print(S)
    print(pp)
    for f in S.newforms('a'):
        H = f.hecke_eigenvalue_field()
        if H is QQ:
            ps = [N]
            HNs = [GF(N)]
        else:
            ps = H.primes_above(pp)
            HNs = [p.residue_field() for p in ps]
        print("primes comp")
        for p,HN in zip(ps, HNs):
            print(p,HN)
            for l in prime_range(100):
                if l == N:
                    continue
                if HN(f[l]) != 1 + l^(k-1):
                    break
            else:
                return (p,f)
p,f = find_ribet(37, 32)
f

6 11
Cuspidal subspace of dimension 2 of Modular Forms space of dimension 4, character [-zeta6 + 1] and weight 2 over Cyclotomic Field of order 6 and degree 2
Fractional ideal (-7*zeta6 + 4)
primes comp
Fractional ideal ((-zeta6 - 1)*a0 + 2*zeta6 - 2) Residue field of Fractional ideal ((-zeta6 - 1)*a0 + 2*zeta6 - 2)


q + a0*q^2 + ((zeta6 - 2)*a0 - zeta6 + 1)*q^3 + (zeta6 - 1)*q^4 + (-2*zeta6*a0 - zeta6 - 1)*q^5 + O(q^6)

In [45]:
p

Fractional ideal ((-zeta6 - 1)*a0 + 2*zeta6 - 2)

We can try and find this form on the lmbfdb now by looking at traces and comparing with
https://www.lmfdb.org/ModularForm/GL2/Q/holomorphic/?level=37&weight=2&search_type=List

In [11]:
print(f[2].trace())
print(f[3].trace())
print(f[5].trace())
print(f[7].trace())

0
2
-6
0


So this is
https://www.lmfdb.org/ModularForm/GL2/Q/holomorphic/37/2/e/a/

Actually we  can tell this just from the character alone.
Just from searching for the right character orbit https://www.lmfdb.org/ModularForm/GL2/Q/holomorphic/?count=None&hst=List&weight=2&char_label=37.11&search_type=List

In [46]:
f.hecke_eigenvalue_field()

Number Field in a0 with defining polynomial x^2 + zeta6 - 1 over its base field

Question:
---------

Can one find any other examples in this way? Seems hard!

Here is some other random code:

In [ ]:
N = 100
for k in range(2,N,2):
    for p,_ in bernoulli(k).numerator().factor():
        if k > p - 3:
            continue
        print(p,k)
        print((p-1)%(k-2))

In [ ]:
for p in prime_range(3,70000):
    for (k,i) in enumerate(bernoulli_mod_p(p)):
        if i != 0:
            continue
        #print(p, 2*k)
        if (p-1)/gcd(p-1, 2*k-2) <= 20:
            print((p-1)/gcd(p-1, 2*k-2))
            print(euler_phi((p-1)/gcd(p-1, 2*k-2)))
            print(p,2*k)

In [14]:
find_ribet(691, 12)

69 316
Cuspidal subspace of dimension 57 of Modular Forms space of dimension 59, character [-zeta69^38 - zeta69^15] and weight 2 over Cyclotomic Field of order 69 and degree 44


KeyboardInterrupt: 

In [14]:
for p in prime_range(7,10000000):
    if bernoulli_mod_p_single(p, p  -3 ) == 0:
        print(p)

16843
2124679


In [43]:
find_ribet(617, 338)

Fractional ideal (617, zeta11 + 128) 11
1
[(0, 0), (1, 1), (1, 351), (1, 392), (1, 418), (1, 31), (1, 1), (1, 351), (1, 489), (1, 31), (1, 392), (1, 344), (1, 351), (1, 31), (1, 418), (1, 429), (1, 113), (1, 31), (1, 392), (1, 113), (1, 1), (1, 1), (1, 429), (1, 175), (1, 418), (1, 344), (1, 392), (1, 429), (1, 489), (1, 489), (1, 31), (1, 392), (1, 175), (1, 342), (1, 392), (1, 392), (1, 1), (1, 489), (1, 175), (1, 429), (1, 351), (1, 113), (1, 351), (1, 342), (1, 31), (1, 344), (1, 342), (1, 418), (1, 489), (1, 418), (1, 429), (1, 429), (1, 1), (1, 489), (1, 31), (1, 175), (1, 113), (1, 489), (1, 113), (1, 31), (1, 392), (1, 418), (1, 1), (1, 392), (1, 342), (1, 344), (1, 344), (1, 113), (1, 1), (1, 113), (1, 1), (1, 392), (1, 351), (1, 489), (1, 113), (1, 342), (1, 342), (1, 429), (1, 31), (1, 175), (1, 418), (1, 344), (1, 175), (1, 392), (1, 418), (1, 344), (1, 344), (1, 418), (1, 392), (1, 392), (1, 429), (1, 392), (1, 344), (1, 31), (1, 489), (1, 418), (1, 113), (1, 392), (1, 489

[(0, 0), (1, 1), (175, 351), (342, 392), (392, 418), (351, 31), (1, 1), (175, 351), (113, 489), (351, 31), (342, 392), (418, 344), (175, 351), (351, 31), (392, 418), (344, 429), (31, 113), (351, 31), (342, 392), (31, 113), (1, 1), (1, 1), (344, 429), (489, 175), (392, 418), (418, 344), (342, 392), (344, 429), (113, 489), (113, 489), (351, 31), (342, 392), (489, 175), (429, 342), (342, 392), (342, 392), (1, 1), (113, 489), (489, 175), (344, 429), (175, 351), (31, 113), (175, 351), (429, 342), (351, 31), (418, 344), (429, 342), (392, 418), (113, 489), (392, 418), (344, 429), (344, 429), (1, 1), (113, 489), (351, 31), (489, 175), (31, 113), (113, 489), (31, 113), (351, 31), (342, 392), (392, 418), (1, 1), (342, 392), (429, 342), (418, 344), (418, 344), (31, 113), (1, 1), (31, 113), (1, 1), (342, 392), (175, 351), (113, 489), (31, 113), (429, 342), (429, 342), (344, 429), (351, 31), (489, 175), (392, 418), (418, 344), (489, 175), (342, 392), (392, 418), (418, 344), (418, 344), (392, 418), 

[(0, 0), (1, 1), (344, 351), (113, 392), (489, 418), (429, 31), (1, 1), (344, 351), (392, 489), (429, 31), (113, 392), (175, 344), (344, 351), (429, 31), (489, 418), (351, 429), (342, 113), (429, 31), (113, 392), (342, 113), (1, 1), (1, 1), (351, 429), (418, 175), (489, 418), (175, 344), (113, 392), (351, 429), (392, 489), (392, 489), (429, 31), (113, 392), (418, 175), (31, 342), (113, 392), (113, 392), (1, 1), (392, 489), (418, 175), (351, 429), (344, 351), (342, 113), (344, 351), (31, 342), (429, 31), (175, 344), (31, 342), (489, 418), (392, 489), (489, 418), (351, 429), (351, 429), (1, 1), (392, 489), (429, 31), (418, 175), (342, 113), (392, 489), (342, 113), (429, 31), (113, 392), (489, 418), (1, 1), (113, 392), (31, 342), (175, 344), (175, 344), (342, 113), (1, 1), (342, 113), (1, 1), (113, 392), (344, 351), (392, 489), (342, 113), (31, 342), (31, 342), (351, 429), (429, 31), (418, 175), (489, 418), (175, 344), (418, 175), (113, 392), (489, 418), (175, 344), (175, 344), (489, 418)

11 429
Cuspidal subspace of dimension 50 of Modular Forms space of dimension 52, character [zeta11^7] and weight 2 over Cyclotomic Field of order 11 and degree 10


KeyboardInterrupt: 

In [48]:
find_ribet(8849, 6638)

4 2994
Cuspidal subspace of dimension 736 of Modular Forms space of dimension 738, character [zeta4] and weight 2 over Cyclotomic Field of order 4 and degree 2


KeyboardInterrupt: 

In [27]:
len(DirichletGroup(31, zeta_order=5, base_ring=CyclotomicField(5)))

5

In [50]:
find_ribet(881, 162)

11 237
Cuspidal subspace of dimension 72 of Modular Forms space of dimension 74, character [-zeta11^9 - zeta11^8 - zeta11^7 - zeta11^6 - zeta11^5 - zeta11^4 - zeta11^3 - zeta11^2 - zeta11 - 1] and weight 2 over Cyclotomic Field of order 11 and degree 10


KeyboardInterrupt: 

In [53]:
find_ribet(401,382)

20 179
Cuspidal subspace of dimension 32 of Modular Forms space of dimension 34, character [zeta20^3] and weight 2 over Cyclotomic Field of order 20 and degree 8


KeyboardInterrupt: 

In [60]:
find_ribet(59, 44)

29 22
Cuspidal subspace of dimension 4 of Modular Forms space of dimension 6, character [zeta29^13] and weight 2 over Cyclotomic Field of order 29 and degree 28
Fractional ideal (59, zeta29 + 2)


KeyboardInterrupt: 

In [56]:
find_ribet(101,68)

50 70
Cuspidal subspace of dimension 8 of Modular Forms space of dimension 10, character [zeta50^17] and weight 2 over Cyclotomic Field of order 50 and degree 20


KeyboardInterrupt: 

In [59]:
find_ribet(103,64)

51 16
Cuspidal subspace of dimension 8 of Modular Forms space of dimension 10, character [-zeta51^20 - zeta51^3] and weight 2 over Cyclotomic Field of order 51 and degree 32
Fractional ideal (103, zeta51 + 6)


KeyboardInterrupt: 

In [5]:
SS = f.parent();SS

Cuspidal subspace of dimension 2 of Modular Forms space of dimension 4, character [-zeta6 + 1] and weight 2 over Number Field in a0 with defining polynomial x^2 + zeta6 - 1 over its base field

In [8]:
SS.basis()

[
q + (-zeta6 + 1)*q^3 + (zeta6 - 1)*q^4 + (-zeta6 - 1)*q^5 + O(q^6),
q^2 + (zeta6 - 2)*q^3 - 2*zeta6*q^5 + O(q^6)
]

In [7]:
SS.change_ring(QQ)

ValueError: Space cannot be defined over Rational Field

In [14]:
f

q + a0*q^2 + ((zeta6 - 2)*a0 - zeta6 + 1)*q^3 + (zeta6 - 1)*q^4 + (-2*zeta6*a0 - zeta6 - 1)*q^5 + O(q^6)

In [11]:
Af  = AbelianVariety(f); Af

Newform abelian subvariety 37aG1 of dimension 2 of J1(37)

In [13]:
Af.dimension()

2

In [15]:
Af.conductor()

ArithmeticError: self and other must have the same ambient space.

In [19]:
f.base_ring()

Number Field in a0 with defining polynomial x^2 + zeta6 - 1 over its base field

In [21]:
CuspForms?

In [24]:
f

q + a0*q^2 + ((zeta6 - 2)*a0 - zeta6 + 1)*q^3 + (zeta6 - 1)*q^4 + (-2*zeta6*a0 - zeta6 - 1)*q^5 + O(q^6)

In [28]:
def find_ribet(N,k):
    S = CuspForms(Gamma1(N))
    for f in S.newforms('a'):
        H = f.hecke_eigenvalue_field()
        if H is QQ:
            ps = [N]
            HNs = [GF(N)]
        else:
            ps = H.primes_above(N)
            HNs = [p.residue_field() for p in ps]
        for p,HN in zip(ps, HNs):
            for l in prime_range(100):
                if l == 37:
                    continue
                if HN(f[l]) != 1 + l^(k-1):
                    break
            else:
                return (p,f)
p,f = find_ribet(37, 32)
f

q + a4*q^2 + (-a4^3 + a4^2 - a4)*q^3 - a4^2*q^4 + (2*a4^3 + a4^2 - 2*a4 - 2)*q^5 + O(q^6)

In [31]:
Af = AbelianVariety(f); Af

Newform abelian subvariety 37eG1 of dimension 4 of J1(37)

In [36]:
Af.newform_label()

'37eG1'

In [38]:
Af.is_simple()

True

In [41]:
Aff = Af.decomposition()[0]; Aff

Simple abelian subvariety 37eG1(1,37) of dimension 4 of J1(37)

In [43]:
Aff.rank()

8

In [ ]:
from collections import defaultdict
facp = defaultdict(list)
for p in prime_range(2,700):
    if p != 37:
        facp[tuple((P.degree(),e) for P,e in (Aff.frobenius_polynomial(p)).factor())].append(p)
        print(p, Aff.frobenius_polynomial(p))
        print((Aff.frobenius_polynomial(p)).factor())

In [58]:
facp

defaultdict(<class 'list'>, {((8, 1),): [2, 3, 17, 19, 23, 29, 31, 53, 59, 61, 67, 79, 83, 89, 97, 103, 107, 109, 113, 127, 131, 139, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199, 227, 239, 241, 251, 257, 263, 271, 277, 281, 293, 311, 317, 331, 337, 347, 349, 353, 367, 373, 379, 383, 389, 401, 419, 421, 431, 439, 449, 457, 461, 463, 467, 479, 487, 499, 503, 509, 521, 523, 541, 547, 557, 563, 569, 571, 577, 587, 599, 601, 607, 613, 617, 631, 641, 643, 647, 653, 659, 661, 673, 683, 691], ((2, 2), (4, 1)): [5], ((4, 1), (4, 1)): [7], ((4, 2),): [11, 13, 41, 43, 47, 71, 137, 149, 223, 233, 269, 307, 313, 359, 397, 409, 443, 491, 593, 619, 677], ((2, 4),): [73, 101, 211, 433], ((2, 2), (2, 2)): [229, 283]})

In [54]:
oeis.find_by_subsequence([11, 13, 41, 43, 47, 71, 137, 149, 223, 233, 269])

In [55]:
[11, 13, 41, 43, 47, 71, 137, 149, 223, 233, 269]+ [73, 101, 211]+ [229, 283]

[11, 13, 41, 43, 47, 71, 137, 149, 223, 233, 269, 73, 101, 211, 229, 283]

In [56]:
sorted(_)

[11, 13, 41, 43, 47, 71, 73, 101, 137, 149, 211, 223, 229, 233, 269, 283]

In [59]:
f

q + a4*q^2 + (-a4^3 + a4^2 - a4)*q^3 - a4^2*q^4 + (2*a4^3 + a4^2 - 2*a4 - 2)*q^5 + O(q^6)

In [61]:
f.abelian_variety()

Newform abelian subvariety 37eG1 of dimension 4 of J1(37)

In [62]:
f.modsym_eigenspace()

Vector space of degree 115 and dimension 2 over Number Field in a4 with defining polynomial x^4 - x^2 + 1
Basis matrix:
2 x 115 dense matrix over Number Field in a4 with defining polynomial x^4 - x^2 + 1

In [64]:
f.period??

In [66]:
m = f.group()(matrix([[1, 37], [0, 1]]))

In [67]:
f.period(m)

NotImplementedError: period pairing only implemented for cusp forms of trivial character

In [92]:
f.character()

Dirichlet character modulo 37 of conductor 37 mapping 2 |--> a4^2

In [93]:
_.base_extend(CC)

TypeError: no coercion map from Number Field in a4 with defining polynomial x^4 - x^2 + 1 to Complex Field with 53 bits of precision is defined

In [96]:
for chi in DirichletGroup(37):
    if chi.conrey_number() == 11:
        break

In [253]:
SS = CuspForms(chi,prec=400);SS

Cuspidal subspace of dimension 2 of Modular Forms space of dimension 4, character [-zeta6 + 1] and weight 2 over Cyclotomic Field of order 6 and degree 2

In [254]:
ff,gg= SS.basis(); ff,gg

(q + (-zeta6 + 1)*q^3 + (zeta6 - 1)*q^4 + (-zeta6 - 1)*q^5 + (2*zeta6 - 1)*q^6 - zeta6*q^9 - 2*q^10 - 3*q^11 + zeta6*q^12 + (2*zeta6 + 2)*q^13 + (-4*zeta6 + 2)*q^14 + (-zeta6 + 2)*q^15 + zeta6*q^16 + (-zeta6 + 2)*q^17 + (2*zeta6 + 2)*q^18 + (-zeta6 - 1)*q^19 + (-zeta6 + 2)*q^20 + 6*zeta6*q^21 + (-zeta6 + 2)*q^22 + (-2*zeta6 + 1)*q^23 + (-3*zeta6 - 3)*q^24 + 2*zeta6*q^25 - 4*q^27 + (-2*zeta6 + 1)*q^29 + (-zeta6 + 1)*q^30 + (-6*zeta6 + 3)*q^31 + (6*zeta6 - 6)*q^33 + (2*zeta6 - 2)*q^34 + (4*zeta6 - 8)*q^35 + q^36 + (7*zeta6 - 3)*q^37 - 3*q^38 + (-2*zeta6 + 4)*q^39 + (-6*zeta6 + 6)*q^40 + (3*zeta6 - 3)*q^41 + (-2*zeta6 - 2)*q^42 + (4*zeta6 - 2)*q^43 + (-3*zeta6 + 3)*q^44 + (-6*zeta6 + 3)*q^45 - zeta6*q^46 + 3*q^47 + q^48 - 5*zeta6*q^49 + (4*zeta6 + 4)*q^50 + (-6*zeta6 + 3)*q^51 + (2*zeta6 - 4)*q^52 + 6*zeta6*q^53 + (zeta6 + 1)*q^55 + (6*zeta6 + 6)*q^56 + (-2*zeta6 + 4)*q^57 - 2*zeta6*q^58 + (2*zeta6 - 4)*q^59 + (2*zeta6 - 1)*q^60 + (-zeta6 - 1)*q^61 + 3*zeta6*q^62 + 12*q^63 - 7*q^64 - 6*ze

In [255]:
si = SS.base_ring().galois_group().gen()

In [256]:
ff1 = ff.q_expansion()
ff2 = ff.q_expansion().map_coefficients(si)
gg1 = gg.q_expansion()
gg2 = gg.q_expansion().map_coefficients(si)

In [257]:
L = []
for a,b,c,d in Compositions(4, length=4, min_part=0):
    print(a,b,c,d)
    t = ff1^a*ff2^b*gg1^c*gg2^d
    L.append([(t)[n] for n in range(0,399)])

4 0 0 0
3 1 0 0
3 0 1 0
3 0 0 1
2 2 0 0
2 1 1 0
2 1 0 1
2 0 2 0
2 0 1 1
2 0 0 2
1 3 0 0
1 2 1 0
1 2 0 1
1 1 2 0
1 1 1 1
1 1 0 2
1 0 3 0
1 0 2 1
1 0 1 2
1 0 0 3
0 4 0 0
0 3 1 0
0 3 0 1
0 2 2 0
0 2 1 1
0 2 0 2
0 1 3 0
0 1 2 1
0 1 1 2
0 1 0 3
0 0 4 0
0 0 3 1
0 0 2 2
0 0 1 3
0 0 0 4


In [258]:
matrix(L).kernel()

Vector space of degree 35 and dimension 4 over Cyclotomic Field of order 6 and degree 2
Basis matrix:
[                     0                      0                      1                      0                      0                      0                      0                      0                      0                      0                      0                      0    -12/37*zeta6 - 9/37                      0                      0    -33/37*zeta6 + 3/37             -zeta6 + 1                      0                      0    28/37*zeta6 - 16/37                      0    12/37*zeta6 - 28/37                      0                      0     3/37*zeta6 + 30/37                      0                      0                      0    84/37*zeta6 + 63/37                      0                      0                      0                      0                      0                      0]
[                     0                      0                      0                      

In [259]:
P.<x1,x2,x3,x4> = ProjectiveSpace(3,ff.base_ring())

In [263]:
matrix(L).kernel().basis()

[
(0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, -12/37*zeta6 - 9/37, 0, 0, -33/37*zeta6 + 3/37, -zeta6 + 1, 0, 0, 28/37*zeta6 - 16/37, 0, 12/37*zeta6 - 28/37, 0, 0, 3/37*zeta6 + 30/37, 0, 0, 0, 84/37*zeta6 + 63/37, 0, 0, 0, 0, 0, 0),
(0, 0, 0, 1, 0, -3/4*zeta6 + 3/4, 0, 0, 0, 0, 0, 0, 0, -9/14*zeta6 + 9/28, 0, 0, 0, 3*zeta6 - 3, 0, 0, 0, 0, 3/4*zeta6 - 7/4, 0, 0, -3/28*zeta6 - 15/14, 4/7*zeta6, 0, 0, -4/7*zeta6 - 3/7, 0, 0, 0, 0, 0),
(0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 28/37*zeta6 - 16/37, 1/3*zeta6 - 2/3, 0, 0, -40/111*zeta6 + 44/111, 0, 0, 0, 0, -28/37*zeta6 - 21/37, 0, 0, 0, -77/37*zeta6 + 7/37, 0, 0, 0, 0, 0, 0),
(0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 4/7*zeta6 - 4/7, 0, 0, 0, -zeta6 + 2, 0, 0, 0, 0, 0, 0, 0, -4/7*zeta6 - 3/7, -8/21*zeta6 + 4/21, 0, 0, 11/21*zeta6 - 1/21, 0, 0, 0, 0, 0)
]

In [261]:
ccs = []
for ba in matrix(L).kernel().basis():
    cc = x1.parent()(0)
    for i,(a,b,c,d) in enumerate(Compositions(4, length=4, min_part=0)):
        cc += ba[i]*x1^a*x2^b*x3^c*x4^d
    ccs.append(cc)

In [262]:
ccs

[x1^3*x3 + (12/37*zeta6 - 28/37)*x2^3*x3 + (-zeta6 + 1)*x1*x3^3 + (-12/37*zeta6 - 9/37)*x1*x2^2*x4 + (3/37*zeta6 + 30/37)*x2^2*x3*x4 + (-33/37*zeta6 + 3/37)*x1*x2*x4^2 + (84/37*zeta6 + 63/37)*x2*x3*x4^2 + (28/37*zeta6 - 16/37)*x1*x4^3,
 (-3/4*zeta6 + 3/4)*x1^2*x2*x3 + (-9/14*zeta6 + 9/28)*x1*x2*x3^2 + (4/7*zeta6)*x2*x3^3 + x1^3*x4 + (3/4*zeta6 - 7/4)*x2^3*x4 + (3*zeta6 - 3)*x1*x3^2*x4 + (-3/28*zeta6 - 15/14)*x2^2*x4^2 + (-4/7*zeta6 - 3/7)*x2*x4^3,
 x1^2*x3^2 + (1/3*zeta6 - 2/3)*x1*x3^3 + (-28/37*zeta6 - 21/37)*x2^2*x3*x4 + (28/37*zeta6 - 16/37)*x1*x2*x4^2 + (-77/37*zeta6 + 7/37)*x2*x3*x4^2 + (-40/111*zeta6 + 44/111)*x1*x4^3,
 (4/7*zeta6 - 4/7)*x1*x2*x3^2 + (-8/21*zeta6 + 4/21)*x2*x3^3 + x1^2*x3*x4 + (-zeta6 + 2)*x1*x3^2*x4 + (-4/7*zeta6 - 3/7)*x2^2*x4^2 + (11/21*zeta6 - 1/21)*x2*x4^3]

In [239]:
cc.factor()

((4/7*zeta6 - 4/7)) * (x1*x2*x3^2 + (1/3*zeta6 - 2/3)*x2*x3^3 + (-7/4*zeta6)*x1^2*x3*x4 + (-7/4*zeta6 - 7/4)*x1*x3^2*x4 + (7/4*zeta6 - 1)*x2^2*x4^2 + (-5/6*zeta6 + 11/12)*x2*x4^3)

In [240]:
curv = P.subscheme(ccs)

In [241]:
ccs[0].parent().ideal(ccs)

Ideal (x1^3*x3 + (12/37*zeta6 - 28/37)*x2^3*x3 + (-zeta6 + 1)*x1*x3^3 + (-12/37*zeta6 - 9/37)*x1*x2^2*x4 + (3/37*zeta6 + 30/37)*x2^2*x3*x4 + (-33/37*zeta6 + 3/37)*x1*x2*x4^2 + (84/37*zeta6 + 63/37)*x2*x3*x4^2 + (28/37*zeta6 - 16/37)*x1*x4^3, (-3/4*zeta6 + 3/4)*x1^2*x2*x3 + (-9/14*zeta6 + 9/28)*x1*x2*x3^2 + (4/7*zeta6)*x2*x3^3 + x1^3*x4 + (3/4*zeta6 - 7/4)*x2^3*x4 + (3*zeta6 - 3)*x1*x3^2*x4 + (-3/28*zeta6 - 15/14)*x2^2*x4^2 + (-4/7*zeta6 - 3/7)*x2*x4^3, x1^2*x3^2 + (1/3*zeta6 - 2/3)*x1*x3^3 + (-28/37*zeta6 - 21/37)*x2^2*x3*x4 + (28/37*zeta6 - 16/37)*x1*x2*x4^2 + (-77/37*zeta6 + 7/37)*x2*x3*x4^2 + (-40/111*zeta6 + 44/111)*x1*x4^3, (4/7*zeta6 - 4/7)*x1*x2*x3^2 + (-8/21*zeta6 + 4/21)*x2*x3^3 + x1^2*x3*x4 + (-zeta6 + 2)*x1*x3^2*x4 + (-4/7*zeta6 - 3/7)*x2^2*x4^2 + (11/21*zeta6 - 1/21)*x2*x4^3) of Multivariate Polynomial Ring in x1, x2, x3, x4 over Cyclotomic Field of order 6 and degree 2

In [242]:
_.groebner_basis()

[x1^6*x4 + (33/37*zeta6 - 77/37)*x1^3*x2^3*x4 + (-33/37*zeta6 + 40/37)*x2^6*x4 + (zeta6 + 1)*x1*x3^5*x4 + (294/37*zeta6 - 168/37)*x2^2*x3^3*x4^2 + (144/37*zeta6 + 108/37)*x1^3*x2*x4^3 + (-63/37*zeta6 - 297/37)*x2^4*x4^3 + (860/37*zeta6 - 946/37)*x2*x3^3*x4^3 + (429/37*zeta6 - 39/37)*x1^3*x4^4 + (-564/37*zeta6 - 312/37)*x2^3*x4^4 + (-873/37*zeta6 - 2619/148)*x1^2*x3*x4^4 + (-1381/148*zeta6 - 6905/74)*x1*x3^2*x4^4 + (390/37*zeta6 - 273/148)*x2^2*x4^5 + (-4745/148*zeta6 + 3055/148)*x2*x4^6, x2^4*x3 + (11/9*zeta6 - 10/9)*x2*x3^4 + (7/3*zeta6 - 1)*x1^4*x4 + (-7/3*zeta6)*x1*x2^3*x4 + (10/9*zeta6 - 110/9)*x1*x3^3*x4 + (-2*zeta6 + 1)*x1*x2^2*x4^2 + (-11*zeta6)*x2^2*x3*x4^2 + (4*zeta6 - 4)*x1*x2*x4^3 + (-218/9*zeta6 + 109/9)*x2*x3*x4^3 + (-19/9*zeta6 + 38/9)*x1*x4^4, x2^3*x3^2 + (4/3*zeta6 - 7/3)*x1*x3^4 + (-zeta6)*x1*x2^2*x3*x4 + (-4/3*zeta6 - 4/3)*x2^2*x3^2*x4 + (zeta6 - 1)*x1^2*x2*x4^2 + (-4/3*zeta6 + 2/3)*x1*x2*x3*x4^2 + (-7*zeta6)*x2*x3^2*x4^2 + (-1/3*zeta6 + 2/3)*x1^2*x4^3 + (-4/3*zeta6 +

In [243]:
Curve(curv)

Projective Curve over Cyclotomic Field of order 6 and degree 2 defined by x1^3*x3 + (12/37*zeta6 - 28/37)*x2^3*x3 + (-zeta6 + 1)*x1*x3^3 + (-12/37*zeta6 - 9/37)*x1*x2^2*x4 + (3/37*zeta6 + 30/37)*x2^2*x3*x4 + (-33/37*zeta6 + 3/37)*x1*x2*x4^2 + (84/37*zeta6 + 63/37)*x2*x3*x4^2 + (28/37*zeta6 - 16/37)*x1*x4^3, (-3/4*zeta6 + 3/4)*x1^2*x2*x3 + (-9/14*zeta6 + 9/28)*x1*x2*x3^2 + (4/7*zeta6)*x2*x3^3 + x1^3*x4 + (3/4*zeta6 - 7/4)*x2^3*x4 + (3*zeta6 - 3)*x1*x3^2*x4 + (-3/28*zeta6 - 15/14)*x2^2*x4^2 + (-4/7*zeta6 - 3/7)*x2*x4^3, x1^2*x3^2 + (1/3*zeta6 - 2/3)*x1*x3^3 + (-28/37*zeta6 - 21/37)*x2^2*x3*x4 + (28/37*zeta6 - 16/37)*x1*x2*x4^2 + (-77/37*zeta6 + 7/37)*x2*x3*x4^2 + (-40/111*zeta6 + 44/111)*x1*x4^3, (4/7*zeta6 - 4/7)*x1*x2*x3^2 + (-8/21*zeta6 + 4/21)*x2*x3^3 + x1^2*x3*x4 + (-zeta6 + 2)*x1*x3^2*x4 + (-4/7*zeta6 - 3/7)*x2^2*x4^2 + (11/21*zeta6 - 1/21)*x2*x4^3

In [244]:
list(ideal(*ccs).groebner_basis())

[x1^6*x4 + (33/37*zeta6 - 77/37)*x1^3*x2^3*x4 + (-33/37*zeta6 + 40/37)*x2^6*x4 + (zeta6 + 1)*x1*x3^5*x4 + (294/37*zeta6 - 168/37)*x2^2*x3^3*x4^2 + (144/37*zeta6 + 108/37)*x1^3*x2*x4^3 + (-63/37*zeta6 - 297/37)*x2^4*x4^3 + (860/37*zeta6 - 946/37)*x2*x3^3*x4^3 + (429/37*zeta6 - 39/37)*x1^3*x4^4 + (-564/37*zeta6 - 312/37)*x2^3*x4^4 + (-873/37*zeta6 - 2619/148)*x1^2*x3*x4^4 + (-1381/148*zeta6 - 6905/74)*x1*x3^2*x4^4 + (390/37*zeta6 - 273/148)*x2^2*x4^5 + (-4745/148*zeta6 + 3055/148)*x2*x4^6,
 x2^4*x3 + (11/9*zeta6 - 10/9)*x2*x3^4 + (7/3*zeta6 - 1)*x1^4*x4 + (-7/3*zeta6)*x1*x2^3*x4 + (10/9*zeta6 - 110/9)*x1*x3^3*x4 + (-2*zeta6 + 1)*x1*x2^2*x4^2 + (-11*zeta6)*x2^2*x3*x4^2 + (4*zeta6 - 4)*x1*x2*x4^3 + (-218/9*zeta6 + 109/9)*x2*x3*x4^3 + (-19/9*zeta6 + 38/9)*x1*x4^4,
 x2^3*x3^2 + (4/3*zeta6 - 7/3)*x1*x3^4 + (-zeta6)*x1*x2^2*x3*x4 + (-4/3*zeta6 - 4/3)*x2^2*x3^2*x4 + (zeta6 - 1)*x1^2*x2*x4^2 + (-4/3*zeta6 + 2/3)*x1*x2*x3*x4^2 + (-7*zeta6)*x2*x3^2*x4^2 + (-1/3*zeta6 + 2/3)*x1^2*x4^3 + (-4/3*zeta6

In [245]:
Curve(curv).genus()

RuntimeError: Error raised calling singular function

Exception ignored in: 'sage.libs.singular.function.LibraryCallHandler.handle_call'
Traceback (most recent call last):
  File "/Users/alex/dev-sage2/local/lib/python3.8/site-packages/sage/rings/polynomial/multi_polynomial_ideal.py", line 1535, in genus
    return Integer(genus(self))
RuntimeError: Error raised calling singular function


RuntimeError: error in Singular function call 'genus':
genus does not yet support extension fields
leaving normal.lib::genus

In [217]:
kk= curv.base_ring().prime_above(5).residue_field();kk

Residue field in zeta6bar of Fractional ideal (5)

In [246]:
Curve(curv).change_ring(kk).zeta_series(2,t)

KeyboardInterrupt: 

In [222]:
TT.<t> = PowerSeries(QQ)

TypeError: __init__() takes at least 2 positional arguments (1 given)

In [109]:
CuspForms(Gamma1(37))(ff.q_expansion(75))

TypeError: q-expansion needed to at least precision 229

In [81]:
SS.free_module().basis??

In [101]:
SS.change_ring(QQ)

TypeError: Unable to coerce zeta6 to a rational

In [266]:
le = Aff.endomorphism_ring();le

Endomorphism ring of Simple abelian subvariety 37eG1(1,37) of dimension 4 of J1(37)

In [267]:
le.calculate_generators()

In [268]:
le.gens()

(Abelian variety endomorphism of Simple abelian subvariety 37eG1(1,37) of dimension 4 of J1(37),
 Abelian variety endomorphism of Simple abelian subvariety 37eG1(1,37) of dimension 4 of J1(37),
 Abelian variety endomorphism of Simple abelian subvariety 37eG1(1,37) of dimension 4 of J1(37),
 Abelian variety endomorphism of Simple abelian subvariety 37eG1(1,37) of dimension 4 of J1(37))

In [269]:
_[0]

Abelian variety endomorphism of Simple abelian subvariety 37eG1(1,37) of dimension 4 of J1(37)

In [270]:
_.characteristic_polynomial()

x^8 - 8*x^7 + 28*x^6 - 56*x^5 + 70*x^4 - 56*x^3 + 28*x^2 - 8*x + 1

In [280]:
x = (ff1 + ff2)/(gg1 + gg2);x

q^-1 + 3/2 + 11/4*q + 37/8*q^2 + 103/16*q^3 + 277/32*q^4 + 847/64*q^5 + 2301/128*q^6 + 6055/256*q^7 + 15749/512*q^8 + 41023/1024*q^9 + 105165/2048*q^10 + 276887/4096*q^11 + 691541/8192*q^12 + 1769647/16384*q^13 + 4481309/32768*q^14 + 11186055/65536*q^15 + 27660325/131072*q^16 + 68713503/262144*q^17 + 167558253/524288*q^18 + 410184823/1048576*q^19 + 995596277/2097152*q^20 + 2419619471/4194304*q^21 + 5826192573/8388608*q^22 + 14086373479/16777216*q^23 + 33689777349/33554432*q^24 + 80662192127/67108864*q^25 + 192446753293/134217728*q^26 + 458130381655/268435456*q^27 + 1083945236117/536870912*q^28 + 2572089337967/1073741824*q^29 + 6061978770525/2147483648*q^30 + 14276275563847/4294967296*q^31 + 33515697270117/8589934592*q^32 + 78578882101215/17179869184*q^33 + 183573031537581/34359738368*q^34 + 428978741225015/68719476736*q^35 + 998583726221621/137438953472*q^36 + 2323427552335439/274877906944*q^37 + 5394109229762557/549755813888*q^38 + 12508772948211239/1099511627776*q^39 + 28920635941960

In [281]:
q = x.parent().gen()

In [289]:
y = (x.derivative() / ((gg1+gg2)/q)^3); show(y.truncate(80))

-1/8*q^-5 - 9/16*q^-4 - 43/32*q^-3 - 121/64*q^-2 + 45/64*q^-1 + 277/16 + 20245/256*q + 134631/512*q^2 + 1528801/2048*q^3 + 7860029/4096*q^4 + 37584171/8192*q^5 + 169739305/16384*q^6 + 183048937/8192*q^7 + 1520217207/32768*q^8 + 1527521899/16384*q^9 + 5968050791/32768*q^10 + 181974235203/524288*q^11 + 678485310595/1048576*q^12 + 2480226437073/2097152*q^13 + 8906287708827/4194304*q^14 + 15733789422383/4194304*q^15 + 27385674464519/4194304*q^16 + 188079636396123/16777216*q^17 + 637748886202489/33554432*q^18 + 4274653811806613/134217728*q^19 + 14170371089495769/268435456*q^20 + 46497524003654439/536870912*q^21 + 151120420861010397/1073741824*q^22 + 60844106180598417/268435456*q^23 + 777282631032585325/2147483648*q^24 + 1231309966195518161/2147483648*q^25 + 3871577392521019263/4294967296*q^26 + 48343544648570239191/34359738368*q^27 + 149883218781803330367/68719476736*q^28 + 461671539735780633645/137438953472*q^29 + 1413217490401570392815/274877906944*q^30 + 2150161479261588280209/274877906944*q^31 + 1626413279753681857413/137438953472*q^32 + 19576816372594238271489/1099511627776*q^33 + 58603137864321424091563/2199023255552*q^34 + 349097850573755248629513/8796093022208*q^35 + 1034772724309569394976437/17592186044416*q^36 + 3052972135682701392068579/35184372088832*q^37 + 8967150492313943444674257/70368744177664*q^38 + 6556153028518793557618635/35184372088832*q^39 + 38187552689540744922427779/140737488355328*q^40 + 13845992257109309202979263/35184372088832*q^41 + 1250183482406795768512909/2199023255552*q^42 + 1842491555934033678810629419/2251799813685248*q^43 + 5284223814463254941565167163/4503599627370496*q^44 + 15101491933202049429958593353/9007199254740992*q^45 + 43009855514105111565874263683/18014398509481984*q^46 + 61043548449214571989396633299/18014398509481984*q^47 + 86358919612577486558830151837/18014398509481984*q^48 + 487158429279451115681849830919/72057594037927936*q^49 + 1369862295703692177025197634941/144115188075855872*q^50 + 7681102395995525857445858921533/576460752303423488*q^51 + 21472607710088204951645561779921/1152921504606846976*q^52 + 59858206197978546457427432342367/2305843009213693952*q^53 + 166407121303713517308504497318661/4611686018427387904*q^54 + 28836310793289594910728177487673/576460752303423488*q^55 + 637949054161475205370588571107257/9223372036854775808*q^56 + 879849978136554792675747394831191/9223372036854775808*q^57 + 2420940953259553258294018103094225/18446744073709551616*q^58 + 26580813041082563802310919769679167/147573952589676412928*q^59 + 72788937950211460468582451547219319/295147905179352825856*q^60 + 198865361567829825577395522613316901/590295810358705651712*q^61 + 542091040103241287409089502965040471/1180591620717411303424*q^62 + 737218915549759421410533075092390133/1180591620717411303424*q^63 + 15631580971082700295269424342031663/18446744073709551616*q^64 + 5418906950027072873607430291510773357/4722366482869645213696*q^65 + 14645777918171173985878466416802664431/9444732965739290427392*q^66 + 79006704224502258502600347115893324081/37778931862957161709568*q^67 + 212678882593456144625983629751165062957/75557863725914323419136*q^68 + 571401334170065968958686389621265399195/151115727451828646838272*q^69 + 1532256690561602285605473108593069516409/302231454903657293676544*q^70 + 1025301468634764074600586882023817760365/151115727451828646838272*q^71 + 5478570033892139922209874774151094147855/604462909807314587353088*q^72 + 3652698112819998162918875197009490322673/302231454903657293676544*q^73 + 9724248876213247719553799016272643549369/604462909807314587353088*q^74 + 206746619752535825952712501488766172946707/9671406556917033397649408*q^75 + 548522897943573554197454885829464033091827/19342813113834066795298816*q^76 + 1452876010221430308547627805044552023710145/38685626227668133590597632*q^77 + 3841960071342118237549319806044340261945195/77371252455336267181195264*q^78 + 5071662098902656789619918444889496572433911/77371252455336267181195264*q^79

In [292]:
matrix(L).kernel().basis()

[

]

In [301]:
zeta6 = ff1.base_ring().gen()

In [316]:
ff1 - f3*(-zeta6 + 1)

q - 2*q^5 + q^6 - q^9 - 2*q^10 - 3*q^11 + q^12 + 4*q^13 - 2*q^14 + q^15 + q^16 + q^17 + 4*q^18 - 2*q^19 + q^20 + 6*q^21 + q^22 - q^23 - 6*q^24 + 2*q^25 - 4*q^27 - q^29 - 3*q^31 - 4*q^35 + q^36 + 4*q^37 - 3*q^38 + 2*q^39 - 4*q^42 + 2*q^43 - 3*q^45 - q^46 + 3*q^47 + q^48 - 5*q^49 + 8*q^50 - 3*q^51 - 2*q^52 + 6*q^53 + 2*q^55 + 12*q^56 + 2*q^57 - 2*q^58 - 2*q^59 + q^60 - 2*q^61 + 3*q^62 + 12*q^63 - 7*q^64 - 6*q^65 - 4*q^66 + q^68 - 6*q^72 - 10*q^75 + q^76 - 6*q^79 - q^80 + 3*q^83 - 6*q^84 + q^85 + 2*q^87 + 3*q^88 + 3*q^89 - 4*q^90 + 2*q^92 - 12*q^93 + 3*q^94 + 9*q^95 + 5*q^96 + q^97 + 9*q^99 - 2*q^100 - 9*q^101 + 5*q^102 - 4*q^103 - 2*q^105 + 4*q^106 + 3*q^109 + 3*q^110 + 7*q^111 + 2*q^113 + 2*q^116 - 2*q^117 + 4*q^119 + 3*q^120 + q^121 - 6*q^122 + 3*q^123 + 6*q^124 - 5*q^125 - 2*q^126 + 7*q^127 + 4*q^129 - 8*q^130 - 9*q^131 - 6*q^132 - 6*q^133 - q^134 + 8*q^135 - 6*q^136 - 3*q^137 + 2*q^138 + 3*q^139 - 4*q^140 - 12*q^143 - 5*q^147 - 7*q^148 + 9*q^149 + 2*q^150 - 10*q^153 + 6*q^154 + 2*q^1

In [310]:
f1 = (ff1 + ff2)/2; f1

q + 1/2*q^3 - 1/2*q^4 - 3/2*q^5 - 1/2*q^9 - 2*q^10 - 3*q^11 + 1/2*q^12 + 3*q^13 + 3/2*q^15 + 1/2*q^16 + 3/2*q^17 + 3*q^18 - 3/2*q^19 + 3/2*q^20 + 3*q^21 + 3/2*q^22 - 9/2*q^24 + q^25 - 4*q^27 + 1/2*q^30 - 3*q^33 - q^34 - 6*q^35 + q^36 + 1/2*q^37 - 3*q^38 + 3*q^39 + 3*q^40 - 3/2*q^41 - 3*q^42 + 3/2*q^44 - 1/2*q^46 + 3*q^47 + q^48 - 5/2*q^49 + 6*q^50 - 3*q^52 + 3*q^53 + 3/2*q^55 + 9*q^56 + 3*q^57 - q^58 - 3*q^59 - 3/2*q^61 + 3/2*q^62 + 12*q^63 - 7*q^64 - 3*q^65 + 9/2*q^67 - 3*q^70 - 3*q^71 - 9*q^72 - 10*q^75 + 3/2*q^76 + 3*q^77 - 3*q^78 - 9/2*q^79 - 1/2*q^81 + 3/2*q^83 - 6*q^84 + q^85 + 3/2*q^87 + 9/2*q^89 - 2*q^90 + 3/2*q^92 - 9*q^93 + 9/2*q^94 + 9/2*q^95 + 15/2*q^96 + 9/2*q^99 - 2*q^100 - 9*q^101 + 5/2*q^102 + 3*q^106 + 3*q^107 + 2*q^108 + 9/2*q^109 + 3*q^110 + 11/2*q^111 + 3*q^113 - 5/2*q^115 + 3/2*q^116 + 5*q^118 + 3/2*q^120 + q^121 - 6*q^122 + 3/2*q^123 + 9/2*q^124 - 3*q^126 + 7/2*q^127 + 3*q^129 - 6*q^130 - 27/2*q^131 - 3*q^132 - 9*q^133 + 6*q^135 - 3*q^136 - 3*q^137 + 2*q^138 + 3/2

In [312]:
f2 = (gg1 + gg2)/2; f2

q^2 - 3/2*q^3 - q^5 + 1/2*q^6 + 3*q^7 - 3/2*q^8 - 3/2*q^10 + 3/2*q^11 + q^15 - 2*q^17 - 1/2*q^18 - 3/2*q^19 + 2*q^20 - 3*q^22 + 1/2*q^23 + 3/2*q^24 + 3*q^26 + q^29 + 3/2*q^30 - 3/2*q^31 - 5/2*q^32 + 6*q^33 + 3/2*q^34 - 6*q^35 - 3*q^36 - 3/2*q^38 - 6*q^39 + 9/2*q^40 + 3*q^42 - 3/2*q^44 + 2*q^45 + 9/2*q^47 - 3/2*q^48 + q^50 - 5/2*q^51 - 4*q^54 + 3/2*q^55 + 10*q^59 - 1/2*q^60 - 3*q^61 - 3*q^63 - 3*q^66 + 3/2*q^67 + q^68 + q^69 - 6*q^70 + 3*q^72 + 1/2*q^74 + 3*q^75 + 3*q^76 - 9*q^77 + 3*q^78 - 9/2*q^79 + q^80 - 3*q^81 - 3/2*q^82 + 3*q^84 + 1/2*q^87 + 9/2*q^88 - 4*q^89 + 12*q^91 + 1/2*q^92 + 6*q^93 + 3*q^94 - 5*q^96 - 3*q^97 - 5/2*q^98 - 6*q^100 - 3*q^103 - 9*q^104 + 3*q^105 + 3*q^106 - 6*q^107 + 12*q^109 + 3/2*q^110 - 6*q^111 + 3*q^112 + 8*q^113 + 3*q^114 - 9/2*q^115 + q^116 - 6*q^117 - 3*q^118 + 3*q^119 - 9*q^121 - 3/2*q^122 - 3/2*q^124 + 3*q^125 + 12*q^126 + 3*q^128 - 3*q^129 - 3*q^130 - 5*q^131 - 6*q^133 + 9/2*q^134 + 4*q^135 + 3*q^137 + 3*q^140 - 3*q^142 + 3*q^143 - 3*q^144 - 6*q^145 +

In [321]:
f3 = (ff1 - ff2)/(-2*zeta6 + 1) +f4; f3

q^3 - 2*q^5 - q^6 - 2*q^7 + 3*q^8 + 4*q^9 - q^10 + q^11 - 2*q^12 - q^18 - 3*q^19 - 4*q^21 - q^23 + 3*q^24 + 8*q^25 + 2*q^26 + 4*q^28 - 2*q^29 - q^30 + 3*q^31 - 5*q^32 - 4*q^33 - q^34 - 2*q^36 - q^38 - 3*q^40 + 6*q^42 + q^44 - 4*q^45 - 2*q^46 + 3*q^47 - q^48 + 2*q^50 + 5*q^51 + 4*q^53 + 3*q^55 - 2*q^58 + q^60 - 6*q^61 - 6*q^62 - 2*q^63 - 8*q^65 + 6*q^66 - q^67 - 2*q^68 + 2*q^69 + 4*q^70 + 7*q^74 + 2*q^75 + 6*q^77 - 2*q^78 - 9*q^79 - 2*q^80 + 2*q^81 + 3*q^82 - 6*q^83 + 2*q^84 + 4*q^86 + q^87 - 9*q^88 - 6*q^90 + q^92 + 12*q^93 + 10*q^95 + 6*q^97 - 5*q^98 - 14*q^99 - 4*q^100 - 6*q^102 + 6*q^103 + 6*q^104 - 6*q^105 + 6*q^106 + 4*q^107 + q^110 - 10*q^111 + 2*q^112 - 2*q^114 + 3*q^115 + 2*q^116 + 12*q^117 + 2*q^118 - 6*q^119 + 6*q^120 - 6*q^121 - q^122 - 6*q^123 - 3*q^124 - 6*q^125 - 6*q^127 - 6*q^129 - 6*q^130 + 8*q^132 - 9*q^134 + 8*q^135 + 6*q^136 + 2*q^137 + 18*q^139 - 6*q^140 + 6*q^142 + 6*q^143 + 2*q^144 + 4*q^145 + 5*q^147 + 2*q^149 - 9*q^151 - 3*q^152 + 8*q^153 - 6*q^154 + 3*q^155 - 6

In [295]:
gg1

q^2 + (zeta6 - 2)*q^3 - 2*zeta6*q^5 + (-zeta6 + 1)*q^6 + (-2*zeta6 + 4)*q^7 + (3*zeta6 - 3)*q^8 + (4*zeta6 - 2)*q^9 + (-zeta6 - 1)*q^10 + (zeta6 + 1)*q^11 + (-2*zeta6 + 1)*q^12 + q^15 - 2*q^17 - zeta6*q^18 - 3*zeta6*q^19 + 2*q^20 + (-4*zeta6 + 2)*q^21 - 3*q^22 + (-zeta6 + 1)*q^23 + 3*zeta6*q^24 + (8*zeta6 - 4)*q^25 + (2*zeta6 + 2)*q^26 + (4*zeta6 - 2)*q^28 + (-2*zeta6 + 2)*q^29 + (-zeta6 + 2)*q^30 + (3*zeta6 - 3)*q^31 - 5*zeta6*q^32 + (-4*zeta6 + 8)*q^33 + (-zeta6 + 2)*q^34 - 6*q^35 + (-2*zeta6 - 2)*q^36 + (-zeta6 - 1)*q^38 - 6*q^39 + (-3*zeta6 + 6)*q^40 + 6*zeta6*q^42 + (zeta6 - 2)*q^44 + (-4*zeta6 + 4)*q^45 + (-2*zeta6 + 1)*q^46 + (3*zeta6 + 3)*q^47 + (-zeta6 - 1)*q^48 + 2*zeta6*q^50 + (5*zeta6 - 5)*q^51 + (4*zeta6 - 2)*q^53 - 4*q^54 + 3*zeta6*q^55 + (-2*zeta6 + 1)*q^58 + 10*q^59 + (zeta6 - 1)*q^60 - 6*zeta6*q^61 + (-6*zeta6 + 3)*q^62 + (-2*zeta6 - 2)*q^63 + (-8*zeta6 + 4)*q^65 + (6*zeta6 - 6)*q^66 + (-zeta6 + 2)*q^67 + (-2*zeta6 + 2)*q^68 + 2*zeta6*q^69 + (4*zeta6 - 8)*q^70 + 3*q^72

In [306]:
f4 = (gg1 - gg2)/(2*zeta6 - 1) - (ff1 - ff2)/(-2*zeta6 + 1); f4

q^4 - 3*q^5 + q^6 - 2*q^7 + 3*q^8 + 3*q^9 - q^10 + q^11 - q^12 + 2*q^13 - 4*q^14 - q^15 + q^16 - q^17 + q^18 - 4*q^19 - q^20 + 2*q^21 - q^22 - 3*q^23 + 10*q^25 + 2*q^26 + 4*q^28 - 4*q^29 - 2*q^30 - 3*q^31 - 5*q^32 + 2*q^33 + q^34 + 4*q^35 - 2*q^36 + 7*q^37 - q^38 - 2*q^39 - 9*q^40 + 3*q^41 + 4*q^42 + 4*q^43 - 2*q^44 - 10*q^45 - 3*q^46 + 3*q^47 - q^48 - 5*q^49 + 6*q^50 - q^51 + 2*q^52 + 10*q^53 + 4*q^55 + 6*q^56 - 2*q^57 - 4*q^58 + 2*q^59 + 3*q^60 - 7*q^61 - 3*q^62 - 2*q^63 - 14*q^65 - 2*q^66 - 10*q^67 + 2*q^69 + 10*q^70 + 6*q^71 + 6*q^72 + 7*q^74 + 2*q^75 - q^76 + 4*q^78 - 12*q^79 - 4*q^80 + 3*q^81 + 3*q^82 - 3*q^83 + 2*q^84 + 4*q^86 + 2*q^87 - 3*q^88 - 3*q^89 - 10*q^90 + 2*q^92 + 6*q^93 - 3*q^94 + 19*q^95 - 5*q^96 + 8*q^97 - 5*q^98 - 5*q^99 - 4*q^100 - q^102 - 2*q^103 + 6*q^104 - 10*q^105 + 8*q^106 - 2*q^107 - 4*q^108 - 3*q^109 + q^110 - 7*q^111 + 2*q^112 - 2*q^113 - 2*q^114 + 8*q^115 + 3*q^116 + 8*q^117 - 8*q^118 + 2*q^119 + 9*q^120 - 6*q^121 - q^122 - 3*q^123 - 16*q^125 + 2*q^126 + 

In [317]:
x = f3/f4;x

q^-1 + 2 + 6*q + 16*q^2 + 43*q^3 + 116*q^4 + 315*q^5 + 850*q^6 + 2295*q^7 + 6197*q^8 + 16734*q^9 + 45190*q^10 + 122031*q^11 + 329528*q^12 + 889852*q^13 + 2402937*q^14 + 6488837*q^15 + 17522315*q^16 + 47316872*q^17 + 127773441*q^18 + 345036596*q^19 + 931729245*q^20 + 2516021188*q^21 + 6794208354*q^22 + 18346931004*q^23 + 49543649506*q^24 + 133786582937*q^25 + 361274350031*q^26 + 975577319681*q^27 + 2634427566178*q^28 + 7113950336307*q^29 + 19210355235122*q^30 + 51875221334632*q^31 + 140082708288337*q^32 + 378276268641106*q^33 + 1021488927259341*q^34 + 2758406262866616*q^35 + 7448739685741125*q^36 + 20114413040910953*q^37 + 54316519176373035*q^38 + 146675135358744029*q^39 + 396078313903885255*q^40 + 1069561179277223638*q^41 + 2888219516341623795*q^42 + 7799284544165843557*q^43 + 21061016677123396938*q^44 + 56872706844100735042*q^45 + 153577808391764921329*q^46 + 414718140549770440321*q^47 + 1119895757740749078415*q^48 + 3024141906459994062165*q^49 + 8166326380999306469674*q^50 + 22052168

In [322]:
q = x.parent().gen()

In [323]:
y = (x.derivative() / (f4/q)); show(y.truncate(80))

-q^-5 - 3*q^-4 - 2*q^-3 + 27*q^-2 + 209*q^-1 + 1072 + 4650*q + 18319*q^2 + 67808*q^3 + 240166*q^4 + 822950*q^5 + 2748174*q^6 + 8989587*q^7 + 28911332*q^8 + 91672343*q^9 + 287198461*q^10 + 890498409*q^11 + 2736402536*q^12 + 8342611826*q^13 + 25257777897*q^14 + 75995868160*q^15 + 227387034830*q^16 + 676955475647*q^17 + 2006215052453*q^18 + 5920994072849*q^19 + 17408668791102*q^20 + 51006476232808*q^21 + 148967982416987*q^22 + 433784349975349*q^23 + 1259685280852368*q^24 + 3648727484392663*q^25 + 10543562737604792*q^26 + 30399600064773904*q^27 + 87467018343858728*q^28 + 251172734488476989*q^29 + 719951310077841151*q^30 + 2060072645156503599*q^31 + 5885090028132390731*q^32 + 16786252433688180385*q^33 + 47810163541665077263*q^34 + 135983286254955902902*q^35 + 386260351942972108121*q^36 + 1095802378346961521328*q^37 + 3105045873726519314282*q^38 + 8788440074581394893228*q^39 + 24847700374023599039743*q^40 + 70179835740553570089683*q^41 + 198020407194259990774809*q^42 + 558209790145783637213743*q^43 + 1572143277136399685162350*q^44 + 4423949663169597222735603*q^45 + 12438466297243323189262314*q^46 + 34944305333372487890978961*q^47 + 98096247113755106115861143*q^48 + 275174058790045545356741642*q^49 + 771353936998963249549326493*q^50 + 2160739258973524236695099030*q^51 + 6048730023454021564045473104*q^52 + 16921909537088203955704663554*q^53 + 47311584352228076305649903414*q^54 + 132198856814551594462835321106*q^55 + 369180422489482395075437586043*q^56 + 1030406572472918223827263675066*q^57 + 2874388127944057217529610465807*q^58 + 8014132059310666917045021371112*q^59 + 22333098498683690069161597676315*q^60 + 62205623676144636342627897907867*q^61 + 173182882562700120979901707099282*q^62 + 481926827533097979569717747316555*q^63 + 1340491197479741594974114247871265*q^64 + 3726998321387504776007910766856810*q^65 + 10357911449855961395037785951916362*q^66 + 28774522527176728288962041393515348*q^67 + 79904630071948606386711515819584375*q^68 + 221803502226579868707353113141565375*q^69 + 615463130453979617853506750416839166*q^70 + 1707171853772233020222409499080828179*q^71 + 4733672381173211113707432383633733034*q^72 + 13121060517419515508176660016228486916*q^73 + 36357443877063507200837999631240705073*q^74 + 100710589044644267749276477486530526874*q^75 + 278880344774871041184362537150729898391*q^76 + 772013877868865741473984385399644650828*q^77 + 2136486064121928746079340266306535193616*q^78 + 5910796164303279652238916501601439408969*q^79

In [328]:
L = []
L.append([(y^2)[n] for n in range(-10,100)])
L.append([(y)[n] for n in range(-10,100)])
L.append([(x*y)[n] for n in range(-10,100)])
L.append([(x^2*y)[n] for n in range(-10,100)])
L.append([(x^3*y)[n] for n in range(-10,100)])
L.append([(x^4*y)[n] for n in range(-10,100)])
for i in range(11):
    t = x^i
    L.append([t[n] for n in range(-10,100)])

In [333]:
matrix(L).change_ring(CyclotomicField(12)).kernel()

Vector space of degree 17 and dimension 0 over Cyclotomic Field of order 12 and degree 4
Basis matrix:
[]

In [330]:
matrix(L).kernel().basis()

[

]

In [374]:
ff2

q + zeta6*q^3 - zeta6*q^4 + (zeta6 - 2)*q^5 + (-2*zeta6 + 1)*q^6 + (zeta6 - 1)*q^9 - 2*q^10 - 3*q^11 + (-zeta6 + 1)*q^12 + (-2*zeta6 + 4)*q^13 + (4*zeta6 - 2)*q^14 + (zeta6 + 1)*q^15 + (-zeta6 + 1)*q^16 + (zeta6 + 1)*q^17 + (-2*zeta6 + 4)*q^18 + (zeta6 - 2)*q^19 + (zeta6 + 1)*q^20 + (-6*zeta6 + 6)*q^21 + (zeta6 + 1)*q^22 + (2*zeta6 - 1)*q^23 + (3*zeta6 - 6)*q^24 + (-2*zeta6 + 2)*q^25 - 4*q^27 + (2*zeta6 - 1)*q^29 + zeta6*q^30 + (6*zeta6 - 3)*q^31 - 6*zeta6*q^33 - 2*zeta6*q^34 + (-4*zeta6 - 4)*q^35 + q^36 + (-7*zeta6 + 4)*q^37 - 3*q^38 + (2*zeta6 + 2)*q^39 + 6*zeta6*q^40 - 3*zeta6*q^41 + (2*zeta6 - 4)*q^42 + (-4*zeta6 + 2)*q^43 + 3*zeta6*q^44 + (6*zeta6 - 3)*q^45 + (zeta6 - 1)*q^46 + 3*q^47 + q^48 + (5*zeta6 - 5)*q^49 + (-4*zeta6 + 8)*q^50 + (6*zeta6 - 3)*q^51 + (-2*zeta6 - 2)*q^52 + (-6*zeta6 + 6)*q^53 + (-zeta6 + 2)*q^55 + (-6*zeta6 + 12)*q^56 + (2*zeta6 + 2)*q^57 + (2*zeta6 - 2)*q^58 + (-2*zeta6 - 2)*q^59 + (-2*zeta6 + 1)*q^60 + (zeta6 - 2)*q^61 + (-3*zeta6 + 3)*q^62 + 12*q^63 - 7*q^

In [375]:
gg1.

q^2 + (zeta6 - 2)*q^3 - 2*zeta6*q^5 + (-zeta6 + 1)*q^6 + (-2*zeta6 + 4)*q^7 + (3*zeta6 - 3)*q^8 + (4*zeta6 - 2)*q^9 + (-zeta6 - 1)*q^10 + (zeta6 + 1)*q^11 + (-2*zeta6 + 1)*q^12 + q^15 - 2*q^17 - zeta6*q^18 - 3*zeta6*q^19 + 2*q^20 + (-4*zeta6 + 2)*q^21 - 3*q^22 + (-zeta6 + 1)*q^23 + 3*zeta6*q^24 + (8*zeta6 - 4)*q^25 + (2*zeta6 + 2)*q^26 + (4*zeta6 - 2)*q^28 + (-2*zeta6 + 2)*q^29 + (-zeta6 + 2)*q^30 + (3*zeta6 - 3)*q^31 - 5*zeta6*q^32 + (-4*zeta6 + 8)*q^33 + (-zeta6 + 2)*q^34 - 6*q^35 + (-2*zeta6 - 2)*q^36 + (-zeta6 - 1)*q^38 - 6*q^39 + (-3*zeta6 + 6)*q^40 + 6*zeta6*q^42 + (zeta6 - 2)*q^44 + (-4*zeta6 + 4)*q^45 + (-2*zeta6 + 1)*q^46 + (3*zeta6 + 3)*q^47 + (-zeta6 - 1)*q^48 + 2*zeta6*q^50 + (5*zeta6 - 5)*q^51 + (4*zeta6 - 2)*q^53 - 4*q^54 + 3*zeta6*q^55 + (-2*zeta6 + 1)*q^58 + 10*q^59 + (zeta6 - 1)*q^60 - 6*zeta6*q^61 + (-6*zeta6 + 3)*q^62 + (-2*zeta6 - 2)*q^63 + (-8*zeta6 + 4)*q^65 + (6*zeta6 - 6)*q^66 + (-zeta6 + 2)*q^67 + (-2*zeta6 + 2)*q^68 + 2*zeta6*q^69 + (4*zeta6 - 8)*q^70 + 3*q^72

In [364]:
L = []
for a,b,c,d in Compositions(6, length=4, min_part=0):
    print(a,b,c,d)
    t = f1^a*f2^b*f3^c*f4^d
    L.append([(t)[n] for n in range(0,399)])

6 0 0 0
5 1 0 0
5 0 1 0
5 0 0 1
4 2 0 0
4 1 1 0
4 1 0 1
4 0 2 0
4 0 1 1
4 0 0 2
3 3 0 0
3 2 1 0
3 2 0 1
3 1 2 0
3 1 1 1
3 1 0 2
3 0 3 0
3 0 2 1
3 0 1 2
3 0 0 3
2 4 0 0
2 3 1 0
2 3 0 1
2 2 2 0
2 2 1 1
2 2 0 2
2 1 3 0
2 1 2 1
2 1 1 2
2 1 0 3
2 0 4 0
2 0 3 1
2 0 2 2
2 0 1 3
2 0 0 4
1 5 0 0
1 4 1 0
1 4 0 1
1 3 2 0
1 3 1 1
1 3 0 2
1 2 3 0
1 2 2 1
1 2 1 2
1 2 0 3
1 1 4 0
1 1 3 1
1 1 2 2
1 1 1 3
1 1 0 4
1 0 5 0
1 0 4 1
1 0 3 2
1 0 2 3
1 0 1 4
1 0 0 5
0 6 0 0
0 5 1 0
0 5 0 1
0 4 2 0
0 4 1 1
0 4 0 2
0 3 3 0
0 3 2 1
0 3 1 2
0 3 0 3
0 2 4 0
0 2 3 1
0 2 2 2
0 2 1 3
0 2 0 4
0 1 5 0
0 1 4 1
0 1 3 2
0 1 2 3
0 1 1 4
0 1 0 5
0 0 6 0
0 0 5 1
0 0 4 2
0 0 3 3
0 0 2 4
0 0 1 5
0 0 0 6


In [365]:
matrix(L).kernel().basis()

[
(0, 0, 1, 0, -1, 0, 0, 0, 0, 0, -3, 0, 0, 0, 0, 0, 0, 0, 0, 0, -39/4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1649/8, 0, -161/4, 0, 0, 0, 0, 0, 0, 0, -7174773/256, 13588691/1024, -2242313/3584, 10487045/7168, 11543083/7168, -3831955/3584, 251/4, -2685041/1024, -3727927/2048, 20054667/2048, -8445395/4096, -241/16, 0, -1459/8, -7021/8, 29543/32, -172825/64, -754213/256, 1404661/256, 14592511/3584, 24509273/7168, -65163951/7168, 1331577/896, -63018287/7168, 25641843/14336, 167393479/14336, -136960603/14336, 63752351/14336, -21623489/14336, -31832251/28672, -50717865/28672, 80364583/14336, -19323/16, 15, 21242925/28672, -139747651/57344, -38707247/8192, 409307725/57344, -40753875/57344, -2151/32, 0),
(0, 0, 0, 1, 0, 0, 0, 0, 0, 11/2, -1, 0, 0, 0, 0, 0, 0, 0, 0, -5/2, -3/2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1725/8, -11/4, -17/4, 0, 0, 0, 0, 0, 0, 0, -129955/128, -349871/512, 1065993/1792, -8982233/3584, 15077561/3584, -3445745/1792, -271/4, -48371/512, -514509/1024, 1286993/1024, -1660881/

In [366]:
P.<x1,x2,x3,x4> = ProjectiveSpace(3,QQ)

In [368]:
ccs6 = []
for ba in matrix(L).kernel().basis():
    cc = x1.parent()(0)
    for i,(a,b,c,d) in enumerate(Compositions(6, length=4, min_part=0)):
        cc += ba[i]*x1^a*x2^b*x3^c*x4^d
    ccs6.append(cc)

In [372]:
ccs6

[-x1^4*x2^2 - 3*x1^3*x2^3 - 39/4*x1^2*x2^4 - 161/4*x1*x2^5 - 1459/8*x2^6 + x1^5*x3 - 7021/8*x2^5*x3 - 172825/64*x2^4*x3^2 + 14592511/3584*x2^3*x3^3 - 2242313/3584*x1*x2*x3^4 - 63018287/7168*x2^2*x3^4 - 2685041/1024*x1*x3^5 - 21623489/14336*x2*x3^5 + 21242925/28672*x3^6 + 29543/32*x2^5*x4 - 754213/256*x2^4*x3*x4 + 24509273/7168*x2^3*x3^2*x4 + 10487045/7168*x1*x2*x3^3*x4 + 25641843/14336*x2^2*x3^3*x4 - 3727927/2048*x1*x3^4*x4 - 31832251/28672*x2*x3^4*x4 - 139747651/57344*x3^5*x4 + 1404661/256*x2^4*x4^2 - 7174773/256*x1*x2^2*x3*x4^2 - 65163951/7168*x2^3*x3*x4^2 + 11543083/7168*x1*x2*x3^2*x4^2 + 167393479/14336*x2^2*x3^2*x4^2 + 20054667/2048*x1*x3^3*x4^2 - 50717865/28672*x2*x3^3*x4^2 - 38707247/8192*x3^4*x4^2 + 13588691/1024*x1*x2^2*x4^3 + 1331577/896*x2^3*x4^3 + 1649/8*x1^2*x3*x4^3 - 3831955/3584*x1*x2*x3*x4^3 - 136960603/14336*x2^2*x3*x4^3 - 8445395/4096*x1*x3^2*x4^3 + 80364583/14336*x2*x3^2*x4^3 + 409307725/57344*x3^3*x4^3 + 251/4*x1*x2*x4^4 + 63752351/14336*x2^2*x4^4 - 241/16*x1*x3*x4^

In [369]:
G = ideal(ccs6).groebner_basis(); G

Polynomial Sequence with 33 Polynomials in 4 Variables

In [361]:
ideal(ccs).groebner_basis() == ideal(ccs + ccs4).groebner_basis()

False

In [370]:
curv = P.subscheme(ccs6)

In [371]:
Curve(curv).genus()

KeyboardInterrupt: 

In [355]:
CCC = Curve(curv)

In [356]:
CCC.plane_projection()

(Scheme morphism:
   From: Projective Curve over Rational Field defined by -x1^3*x2^2 - 3*x1^2*x2^3 - 39/4*x1*x2^4 + x1^4*x3 - 161/4*x1*x2^3*x3 - 493/8*x2^4*x3 - 411/56*x2^3*x3^2 - 29145/112*x1*x2*x3^3 + 1763/56*x2^2*x3^3 - 2517/64*x1*x3^4 - 48283/224*x2*x3^4 - 95785/896*x3^5 + 493/8*x2^4*x4 - 755/8*x1*x2^2*x3*x4 + 8227/56*x2^3*x3*x4 + 30861/56*x1*x2*x3^2*x4 - 28543/112*x2^2*x3^2*x4 + 4477/32*x1*x3^3*x4 + 123987/224*x2*x3^3*x4 + 214999/896*x3^4*x4 - 503/4*x1*x2^2*x4^2 - 977/7*x2^3*x4^2 - 141/4*x1^2*x3*x4^2 - 7551/28*x1*x2*x3*x4^2 + 26693/112*x2^2*x3*x4^2 - 110*x1*x3^2*x4^2 - 13131/28*x2*x3^2*x4^2 - 86503/448*x3^3*x4^2 - 21/2*x1*x2*x4^3 - 419/28*x2^2*x4^3 + 5/2*x1*x3*x4^3 + 267/2*x2*x3*x4^3 + 2725/56*x3^2*x4^3 - 5/2*x2*x4^4 + 91/8*x3*x4^4, -x1^2*x2^3 - 3*x1*x2^4 + x1^3*x2*x3 - 39/4*x1*x2^3*x3 - 11*x2^4*x3 + 93/28*x2^3*x3^2 - 3991/112*x1*x2*x3^3 - 263/112*x2^2*x3^3 - 121/16*x1*x3^4 - 869/28*x2*x3^4 - 6381/448*x3^5 + 11*x2^4*x4 - 33*x1*x2^2*x3*x4 + 40/7*x2^3*x3*x4 + 2977/56*x1*x2*x3^2*x4 

In [378]:

# q-expansion of newform 37.2.e.a, downloaded from the LMFDB on 29 October 2020.

# We generate the q-expansion using the Hecke eigenvalues a_p at the primes.
# Each a_p is given as a linear combination
# of the following basis for the coefficient ring.
# To create the q-expansion as a power series, type "qexp = make_data()"

def make_data():
    from sage.all import prod, floor, prime_powers, gcd, QQ, primes_first_n, next_prime, RR

    def discrete_log(elts, gens, mod):
        # algorithm 2.2, page 16 of https://arxiv.org/abs/0903.2785
        def table_gens(gens, mod):
            T = [1]
            n = len(gens)
            r = [None]*n
            s = [None]*n
            for i in range(n):
                beta = gens[i]
                r[i] = 1
                N = len(T)
                while beta not in T:
                    for Tj in T[:N]:
                        T.append((beta*Tj) % mod)
                    beta = (beta*gens[i]) % mod
                    r[i] += 1
                s[i] = T.index(beta)
            return T, r, s
        T, r, s = table_gens(gens, mod)
        n = len(gens)
        N = [ prod(r[:j]) for j in range(n) ]
        Z = lambda s: [ (floor(s/N[j]) % r[j]) for j in range(n)]
        return [Z(T.index(elt % mod)) for elt in elts]
    def extend_multiplicatively(an):
        for pp in prime_powers(len(an)-1):
            for k in range(1, (len(an) - 1)//pp + 1):
                if gcd(k, pp) == 1:
                    an[pp*k] = an[pp]*an[k]
    from sage.all import PolynomialRing, NumberField
    R = PolynomialRing(QQ, "x")
    f = R(poly_data)
    K = NumberField(f, "a")
    betas = [K.gens()[0]**i for i in range(len(poly_data))]
    convert_elt_to_field = lambda elt: sum(c*beta for c, beta in zip(elt, betas))
    # convert aps to K elements
    primes = primes_first_n(len(aps_data))
    good_primes = [p for p in primes if not p.divides(level)]
    aps = map(convert_elt_to_field, aps_data)
    if not hecke_ring_character_values:
        # trivial character
        char_values = dict(zip(good_primes, [1]*len(good_primes)))
    else:
        gens = [elt[0] for elt in hecke_ring_character_values]
        gens_values = [convert_elt_to_field(elt[1]) for elt in hecke_ring_character_values]
        char_values = dict([(
            p,prod(g**k for g, k in zip(gens_values, elt)))
            for p, elt in zip(good_primes, discrete_log(good_primes, gens, level))
            ])
    an_list_bound = next_prime(primes[-1])
    an = [0]*an_list_bound
    an[1] = 1
    
    from sage.all import PowerSeriesRing
    PS = PowerSeriesRing(K, "q")
    for p, ap in zip(primes, aps):
        if p.divides(level):
            euler_factor = [1, -ap]
        else:
            euler_factor = [1, -ap, p**(weight - 1) * char_values[p]]
        k = RR(an_list_bound).log(p).floor() + 1
        foo = (1/PS(euler_factor)).padded_list(k)
        for i in range(1, k):
            an[p**i] = foo[i]
    extend_multiplicatively(an)
    return PS(an)
level  =  37
weight  =  2
poly_data  =  [1, 0, -1, 0, 1]

# The basis for the coefficient ring is just the power basis
# in the root of the defining polynomial above.
hecke_ring_character_values  =  [\
[2, [0, 0, 1, 0]]]
aps_data  =  [\
[0, 1, 0, 0],
[0, -1, 1, -1],
[-2, -2, 1, 2],
[0, 2, 0, 2],
[-3, 2, 0, -1],
[4, 0, -2, 0],
[1, -2, 1, 0],
[-2, -3, 1, 3],
[-1, 0, 2, 1],
[-1, 0, 2, 2],
[-3, 0, 6, -3],
[4, 0, -7, 0],
[0, 0, -3, 0],
[2, 0, -4, 0],
[3, 6, 0, -3],
[6, 2, -6, -4],
[-2, 10, -2, 0],
[-2, -6, 1, 6],
[0, 1, 9, 1],
[0, 0, -6, 0],
[0, 0, 0, 0],
[-6, -9, 3, 9],
[3, -3, -3, 6],
[3, -4, 3, 0],
[1, 0, -2, -6],
[-9, 0, 0, 0],
[-4, 0, 8, -6],
[0, -4, 6, -4],
[3, 12, 3, 0],
[2, 8, 2, 0],
[7, -3, -7, 6],
[-9, -5, -9, 0],
[-3, 4, 0, -2],
[3, 9, -3, -18],
[9, 4, 0, -2],
[0, 9, -1, 9],
[-3, -10, 3, 20],
[6, -12, 6, 0],
[8, -4, -4, 4],
[-3, 4, 3, -8],
[2, 0, -4, -2],
[0, 4, -3, 4],
[-3, 0, 6, -1],
[-5, 0, 10, -6],
[-21, 4, 21, -8],
[4, 0, -8, -18],
[-10, 0, 0, 0],
[-11, -18, 0, 9],
[14, -11, -7, 11],
[0, 0, -7, 0],
[3, 16, 0, -8],
[4, -10, 4, 0],
[-4, 12, 2, -12],
[16, 0, -32, -2],
[9, -4, 9, 0],
[0, 6, 18, 6],
[18, 4, 0, -2],
[3, -5, -3, 10],
[2, 12, -1, -12],
[3, -28, 3, 0],
[12, 0, -6, 0],
[0, 8, -15, 8],
[-4, 12, 0, -6],
[-6, 20, -6, 0],
[-9, 0, -9, 0],
[9, -4, -9, 8],
[12, -12, 12, 0],
[0, 6, -19, 6],
[1, 0, -2, 11],
[-15, -4, 15, 8],
[-1, -14, -1, 0],
[6, -12, 0, 6],
[0, -6, 10, -6],
[0, -2, -21, -2],
[-12, -2, 12, 4],
[-16, 2, 8, -2],
[6, -26, -3, 26],
[3, 28, 0, -14],
[6, 0, -12, -16],
[5, 0, 5, 0],
[0, 10, 0, -20],
[-1, 0, 2, 18],
[-26, -11, 13, 11],
[27, 0, 0, 0],
[-4, -18, 2, 18],
[30, -4, 0, 2],
[-28, 8, 14, -8],
[-14, 24, 7, -24],
[2, 20, 2, 0],
[-4, 12, 2, -12],
[-10, 0, 20, 14],
[-10, 22, -10, 0],
[17, 0, -34, 3],
[33, 2, 0, -1],
[18, 3, -9, -3],
[-15, -17, -15, 0],
[-15, -12, 15, 24],
[0, 4, 0, 4],
[22, 3, -11, -3],
[1, 0, -2, -6],
[-3, 0, 6, 9],
[-1, 16, -1, 0],
[9, 0, -18, 19],
[-13, 0, 26, -14],
[9, -13, -9, 26],
[4, -6, 4, 0],
[10, 25, -5, -25],
[-3, -24, 0, 12],
[0, -11, -27, -11],
[-21, 6, 21, -12],
[5, 3, 5, 0],
[7, 6, -7, -12],
[-6, -18, 6, 36],
[-14, -24, 0, 12],
[-1, -21, -1, 0],
[-3, -12, 3, 24],
[5, 0, -10, -3],
[4, -40, -2, 40],
[6, 8, -3, -8],
[0, 2, -6, 2],
[6, 18, -3, -18],
[0, 16, 12, 16],
[-27, 20, 0, -10],
[-4, -38, -4, 0],
[3, 11, -3, -22],
[-8, -8, -8, 0],
[0, 0, 0, -30],
[3, 17, -3, -34],
[-16, -12, 8, 12],
[0, 12, -8, 12],
[-29, -6, 0, 3],
[0, 1, 45, 1],
[-30, -24, 0, 12],
[21, -6, 21, 0],
[21, 10, -21, -20],
[-2, 0, 4, 24],
[0, -26, 3, -26],
[-10, -12, 0, 6],
[0, 8, 0, 0],
[16, -20, -8, 20],
[0, 14, 6, 14],
[0, 18, -18, 18],
[29, 15, -29, -30],
[-8, 16, 4, -16],
[2, 48, 2, 0],
[21, -5, -21, 10],
[7, -30, 7, 0],
[1, 0, -2, 22],
[2, 0, -4, -12],
[-42, -2, 42, 4],
[-15, -24, 0, 12],
[0, 10, -15, 10],
[-10, 27, 5, -27],
[3, -30, 0, 15],
[-2, -3, 1, 3],
[-13, 0, 26, -31],
[11, 0, -22, -15],
[0, -2, 27, -2],
[23, 6, -23, -12],
[-27, -18, 27, 36],
[6, 38, 6, 0],
[18, 8, -18, -16],
[24, -12, -12, 12],
[3, -3, -3, 6],
[-14, -8, -14, 0],
[-45, 7, 45, -14],
[-6, 0, 12, -36],
[-2, -24, -2, 0]]
qexp = make_data();qexp

q + a*q^2 + (-a^3 + a^2 - a)*q^3 - a^2*q^4 + (2*a^3 + a^2 - 2*a - 2)*q^5 + (a^3 - 2*a^2 + 1)*q^6 + (2*a^3 + 2*a)*q^7 - 3*a^3*q^8 + (-4*a^3 + a^2 + 2*a - 1)*q^9 + (a^3 - 2*a - 2)*q^10 + (-a^3 + 2*a - 3)*q^11 + (2*a^3 - a^2 - a + 1)*q^12 + (-2*a^2 + 4)*q^13 + (4*a^2 - 2)*q^14 + (a^2 + a + 1)*q^15 + (-a^2 + 1)*q^16 + (a^2 - 2*a + 1)*q^17 + (a^3 - 2*a^2 - a + 4)*q^18 + (3*a^3 + a^2 - 3*a - 2)*q^19 + (a^2 + 2*a + 1)*q^20 + (4*a^3 - 6*a^2 - 2*a + 6)*q^21 + (a^2 - 3*a + 1)*q^22 + (a^3 + 2*a^2 - 1)*q^23 + (-3*a^3 + 3*a^2 + 3*a - 6)*q^24 + (-8*a^3 - 2*a^2 + 4*a + 2)*q^25 + (-2*a^3 + 4*a)*q^26 - 4*q^27 + (-4*a^3 + 2*a)*q^28 + (2*a^3 + 2*a^2 - 1)*q^29 + (a^3 + a^2 + a)*q^30 + (-3*a^3 + 6*a^2 - 3)*q^31 + (5*a^3 - 5*a)*q^32 + (4*a^3 - 6*a^2 + 4*a)*q^33 + (a^3 - 2*a^2 + a)*q^34 + (-4*a^2 - 6*a - 4)*q^35 + (2*a^3 - 4*a + 1)*q^36 + (-7*a^2 + 4)*q^37 + (a^3 - 2*a - 3)*q^38 + (2*a^2 - 6*a + 2)*q^39 + (3*a^3 + 6*a^2 + 3*a)*q^40 - 3*a^2*q^41 + (-6*a^3 + 2*a^2 + 6*a - 4)*q^42 + (-4*a^2 + 2)*q^43 + (-a^3 + 

In [381]:
s1,s2 = qexp.base_ring().galois_group().gens();s1,s2

((1,2)(3,4), (1,3)(2,4))

In [387]:
N = 400
h1 = qexp.truncate(N)
h2 = qexp.map_coefficients(s1).truncate(N)
h3 = qexp.map_coefficients(s2).truncate(N)
h4 = qexp.map_coefficients(s1*s2).truncate(N)

In [388]:
L = []
for a,b,c,d in Compositions(4, length=4, min_part=0):
    print(a,b,c,d)
    t = h1^a*h2^b*h3^c*h4^d
    L.append([(t)[n] for n in range(0,N - 1)])

4 0 0 0
3 1 0 0
3 0 1 0
3 0 0 1
2 2 0 0
2 1 1 0
2 1 0 1
2 0 2 0
2 0 1 1
2 0 0 2
1 3 0 0
1 2 1 0
1 2 0 1
1 1 2 0
1 1 1 1
1 1 0 2
1 0 3 0
1 0 2 1
1 0 1 2
1 0 0 3
0 4 0 0
0 3 1 0
0 3 0 1
0 2 2 0
0 2 1 1
0 2 0 2
0 1 3 0
0 1 2 1
0 1 1 2
0 1 0 3
0 0 4 0
0 0 3 1
0 0 2 2
0 0 1 3
0 0 0 4


In [389]:
matrix(L).kernel()

Vector space of degree 35 and dimension 4 over Number Field in a with defining polynomial x^4 - x^2 + 1
Basis matrix:
[                                       1                                        0                                        0                                        0                                        0                                        0                                        0                                        0                                        0                                        0                                        0                                        0                                        0                                        0                                        0                                        0   3/37*a^3 + 24/37*a^2 - 33/37*a + 32/37                       -27/37*a^2 - 36/37  -3/37*a^3 - 72/37*a^2 + 33/37*a - 96/37                        27/37*a^2 + 36/37                                       -1                     

In [393]:
P.<x1,x2,x3,x4> = ProjectiveSpace(3,h1.base_ring())

In [394]:
ccs = []
for ba in matrix(L).kernel().basis():
    cc = x1.parent()(0)
    for i,(a,b,c,d) in enumerate(Compositions(4, length=4, min_part=0)):
        cc += ba[i]*x1^a*x2^b*x3^c*x4^d
    ccs.append(cc)

In [399]:
ccs

[x1^4 - x2^4 + (3/37*a^3 + 24/37*a^2 - 33/37*a + 32/37)*x1*x3^3 + (-27/37*a^2 - 36/37)*x2*x3^3 + (-27/37*a^2 - 36/37)*x1*x3^2*x4 + (-3/37*a^3 + 72/37*a^2 + 33/37*a + 96/37)*x2*x3^2*x4 + (-3/37*a^3 - 72/37*a^2 + 33/37*a - 96/37)*x1*x3*x4^2 + (27/37*a^2 + 36/37)*x2*x3*x4^2 + (27/37*a^2 + 36/37)*x1*x4^3 + (3/37*a^3 - 24/37*a^2 - 33/37*a - 32/37)*x2*x4^3,
 x1^3*x2 + (a^3 - 2*a)*x1^2*x2^2 - x1*x2^3 + (-a^3 + 2*a)*x2^4 + (-3/37*a^3 + 24/37*a^2 + 33/37*a + 32/37)*x2*x3^3 + (-6/37*a^3 + 66/37*a)*x1*x3^2*x4 + (16/37*a^3 - 27/37*a^2 - 176/37*a - 36/37)*x2*x3^2*x4 + (-8/37*a^3 + 88/37*a)*x1*x3*x4^2 + (3/37*a^3 - 33/37*a)*x2*x3*x4^2 + (6/37*a^3 - 24/37*a^2 - 66/37*a - 32/37)*x1*x4^3 + (-8/37*a^3 + 27/37*a^2 + 88/37*a + 36/37)*x2*x4^3,
 x1^3*x3 + (27/37*a^3 - 54/37*a - 72/37)*x1^2*x2*x3 + (24/37*a^3 - 48/37*a - 165/37)*x1*x2^2*x3 + (-27/37*a^3 + 54/37*a)*x2^3*x3 + (3/37*a^3 + 24/37*a^2 - 33/37*a + 32/37)*x3^4 + (-48/37*a^3 + 96/37*a + 138/37)*x1^2*x2*x4 + 72/37*x1*x2^2*x4 + (24/37*a^3 - 48/37*a - 1

In [396]:
a = h1.base_ring().gen()

In [405]:
h1

12*a^3*q^399 + (-8*a^3 - 18*a^2 + 4*a + 18)*q^398 + (-14*a^3 + 28*a + 3)*q^397 + (-7*a^3 + 14*a - 9)*q^396 + (-30*a^3 - 27*a^2 + 15*a + 27)*q^395 + (21*a^3 - 4*a^2 - 21*a + 8)*q^394 + (22*a^3 - 8*a^2 + 4)*q^393 + 15*a*q^392 + (-2*a^3 + a^2 + a - 1)*q^391 + (2*a^2 + 6*a + 2)*q^390 + (26*a^3 - 3*a^2 - 26*a + 6)*q^389 + (6*a^3 + a^2 - 6*a - 2)*q^388 + (2*a^2 - 12*a + 2)*q^387 + (10*a^3 - 6*a^2 - 5*a + 6)*q^386 + (6*a^2 + 6*a + 6)*q^385 + (3*a^3 - 6*a^2 + 3)*q^384 + (-2*a^3 + 8*a^2 + 2*a - 16)*q^383 + (6*a^3 - a^2 - 3*a + 1)*q^382 + (10*a^3 - 20*a + 16)*q^381 + (5*a^3 - 10*a - 9)*q^380 + (4*a^3 + 12*a^2 - 2*a - 12)*q^379 + (-16*a^2 + 8)*q^378 + (4*a^3 + 6*a^2 + 4*a)*q^377 + (-9*a^3 - 18*a^2 + 9)*q^376 + (-9*a^3 - a^2 + 9*a + 2)*q^375 + (-5*a^3 + 9*a^2 - 5*a)*q^374 + (-2*a^3 - 21*a^2 - 2*a)*q^373 + (6*a^2 - 12*a + 6)*q^372 + (-12*a^3 + 24*a + 12)*q^371 + (11*a^3 + 14*a^2 - a - 8)*q^370 + (6*a^3 - 12*a + 3)*q^369 + (a^2 + a + 1)*q^368 + (-6*a^3 + 10*a^2 - 6*a)*q^367 + (5*a^3 - 3*a^2 + 5*a)*q

In [408]:
a.parent()

Number Field in a with defining polynomial x^4 - x^2 + 1

```
Curve([Curve over Cyclotomic Field of order 12 and degree 4 defined by
333*x^2 + (-27*a^3 - 216*a^2 + 297*a - 288)*u*z + 4440*x*w + 13468*w^2,
(333*a^3 - 666*a - 888)*u^3 + 1776*x*u*z + (-144*a^3 - 819*a^2 + 1584*a - 1092)*z^3 + 11840*u*z*w + (2880*a^3 + 2304*a^2 - 3168*a - 5376)*x*w^2 + (19200*a^3 + 15360*a^2 - 21120*a - 35840)*w^3])
```

In [432]:
P.<x,u,z,w> = ProjectiveSpace(K13, 3)

In [438]:
C = Curve([333*x^2 + (-27*K13(a)^3 - 216*K13(a)^2 + 297*K13(a) - 288)*u*z + 4440*x*w + 13468*w^2,
(333*K13(a)^3 - 666*K13(a) - 888)*u^3 + 1776*x*u*z + (-144*K13(a)^3 - 819*K13(a)^2 + 1584*K13(a) - 1092)*z^3 + 11840*u*z*w + (2880*K13(a)^3 + 2304*K13(a)^2 - 3168*K13(a) - 5376)*x*w^2 + (19200*K13(a)^3 + 15360*K13(a)^2 - 21120*K13(a) - 35840)*w^3]);
C

Projective Curve over Residue field of Fractional ideal (-a^3 - a - 1) defined by -5*x^2 + 3*u*z - 6*x*w, 2*u^3 - 5*x*u*z - 6*z^3 - 3*u*z*w - 5*x*w^2 - 3*w^3

In [436]:
C.rational_points()

[(0 : 2 : 0 : 1),
 (0 : 5 : 0 : 1),
 (0 : 6 : 0 : 1),
 (4 : 7 : 0 : 1),
 (4 : 8 : 0 : 1),
 (4 : 11 : 0 : 1),
 (6 : 7 : 1 : 1),
 (6 : 8 : 9 : 1),
 (6 : 11 : 3 : 1),
 (8 : 4 : 9 : 1),
 (8 : 10 : 1 : 1),
 (8 : 12 : 3 : 1),
 (9 : 1 : 10 : 1),
 (9 : 3 : 12 : 1),
 (9 : 9 : 4 : 1),
 (11 : 2 : 10 : 1),
 (11 : 5 : 4 : 1),
 (11 : 6 : 12 : 1)]

In [437]:
C.zeta_series?

In [419]:
K = a.parent()

In [420]:
K13 = K.prime_above(13).residue_field()

In [441]:
CuspForms(Gamma1(13))

AttributeError: 'CuspidalSubmodule_g1_Q_with_category' object has no attribute 'abelian_variety'

In [446]:
J1 = AbelianVariety(Gamma1(37)); J1

In [450]:
Ji = J1.decomposition()[2]; Ji

Simple abelian subvariety 37cG1(1,37) of dimension 2 of J1(37)

In [453]:
for p in prime_range(100):
    if p == 37:
        continue
    print(Ji.frobenius_polynomial(p))
    

x^4 + x^3 - x^2 + 2*x + 4
x^4 - 3*x^2 + 9
x^4 + x^3 - 4*x^2 + 5*x + 25
x^4 + 2*x^3 - 3*x^2 + 14*x + 49
x^4 + 4*x^3 + 26*x^2 + 44*x + 121
x^4 - 2*x^3 - 9*x^2 - 26*x + 169
x^4 + 3*x^3 - 8*x^2 + 51*x + 289
x^4 - 6*x^3 + 17*x^2 - 114*x + 361
x^4 + 8*x^3 + 62*x^2 + 184*x + 529
x^4 - 18*x^3 + 139*x^2 - 522*x + 841
x^4 + 20*x^3 + 162*x^2 + 620*x + 961
x^4 - 9*x^3 + 40*x^2 - 369*x + 1681
x^4 - 4*x^3 + 90*x^2 - 172*x + 1849
x^4 - 12*x^3 + 130*x^2 - 564*x + 2209
x^4 - 2*x^3 - 49*x^2 - 106*x + 2809
x^4 - 4*x^3 - 43*x^2 - 236*x + 3481
x^4 + x^3 - 60*x^2 + 61*x + 3721
x^4 - 10*x^3 + 33*x^2 - 670*x + 4489
x^4 + 6*x^3 - 35*x^2 + 426*x + 5041
x^4 + 20*x^3 + 246*x^2 + 1460*x + 5329
x^4 + 10*x^3 + 21*x^2 + 790*x + 6241
x^4 - 12*x^3 + 61*x^2 - 996*x + 6889
x^4 + 7*x^3 - 40*x^2 + 623*x + 7921
x^4 - 14*x^3 + 243*x^2 - 1358*x + 9409


In [459]:
fi = Ji.newform('a'); fi

q + a2*q^2 + (a2 + 1)*q^4 + (-a2 - 1)*q^5 + O(q^6)

In [469]:
a2 = fi.base_ring().gen();a2

a2

In [462]:
s= fi.base_ring().galois_group().gen()

In [483]:
ffi = fi.q_expansion(220)
ggi = fi.q_expansion(220).map_coefficients(s)

In [492]:
ff1 = (ffi + ggi)/2 + 1/2*ff2;ff1

q + q^4 - q^5 - 2*q^7 - 3*q^8 + q^10 - 2*q^11 + 2*q^13 + 2*q^14 + 3*q^18 + 6*q^19 - 4*q^23 - 2*q^26 + 9*q^29 - 10*q^31 - 5*q^32 - 3*q^34 + 3*q^36 - 7*q^37 - 6*q^38 + 3*q^40 + 9*q^41 + 2*q^43 - 2*q^44 - 3*q^45 + 6*q^47 + 4*q^50 + 2*q^55 + 6*q^56 - q^61 - 6*q^63 + 7*q^64 + 10*q^67 - 3*q^68 - 2*q^70 - 6*q^71 - 10*q^73 + 3*q^74 + 4*q^77 - 10*q^79 - q^80 - 9*q^81 - 9*q^82 + 3*q^85 + 6*q^88 - 4*q^92 + 7*q^97 + 3*q^98 + 4*q^100 - 3*q^101 + 6*q^103 - 6*q^104 + 2*q^106 + 18*q^107 - 2*q^110 - 2*q^112 + 4*q^115 + 9*q^116 + 6*q^117 + 4*q^118 + 6*q^119 - 7*q^121 + q^122 - 10*q^124 - 9*q^125 + 2*q^130 - 10*q^134 - 9*q^137 - 2*q^140 + 6*q^142 - 4*q^143 - 3*q^144 - 9*q^145 - 4*q^148 - 11*q^149 - 4*q^151 - 18*q^152 + 9*q^153 - 4*q^154 + 10*q^155 + 10*q^158 + 8*q^161 + 9*q^162 + 12*q^166 - 12*q^167 + 18*q^171 + 2*q^172 - 8*q^175 - 7*q^178 + 12*q^179 - 3*q^180 - 5*q^181 + 4*q^182 + 12*q^184 + 4*q^185 + 6*q^188 + 6*q^190 - 4*q^191 + 19*q^193 + 3*q^196 - 6*q^198 - 10*q^199 - 18*q^203 + 2*q^208 - 12*q^209 +

In [493]:
ff2 = (ffi - ggi)/(2*a2 + 1);ff2

q^2 + q^4 - q^5 - 2*q^7 - 3*q^9 + 2*q^13 - q^16 + 3*q^17 + 3*q^18 + 6*q^19 - q^20 - 2*q^22 - 4*q^25 - 2*q^28 - 5*q^32 - 3*q^34 + 2*q^35 - 3*q^37 + 3*q^40 + 9*q^41 - 2*q^44 - 4*q^46 - 3*q^49 + 4*q^50 + 2*q^52 - 2*q^53 + 2*q^55 + 6*q^56 + 9*q^58 - 4*q^59 - q^61 - 10*q^62 - 2*q^65 + 10*q^67 - 2*q^70 - 6*q^71 + 9*q^72 - 4*q^74 + 6*q^76 + 4*q^77 - 10*q^79 - 9*q^81 - 12*q^83 + 2*q^86 + 7*q^89 - 3*q^90 - 4*q^91 - 4*q^92 + 6*q^94 - 6*q^95 + 3*q^98 + 6*q^99 - 6*q^104 + 2*q^106 + 18*q^107 + 5*q^109 + 6*q^113 + 4*q^115 + 9*q^116 + 4*q^118 - 10*q^124 - 6*q^126 - 8*q^127 - 3*q^128 + 2*q^130 - 12*q^133 - 9*q^136 + 16*q^139 - 4*q^143 - 3*q^144 - 9*q^145 - 10*q^146 - 7*q^148 - 4*q^151 - 18*q^152 + 9*q^153 + 10*q^155 + 17*q^157 + 5*q^160 + 8*q^161 - 6*q^163 + 9*q^164 + 12*q^166 - 12*q^167 - 9*q^169 + 3*q^170 + 2*q^172 + 21*q^173 + 2*q^176 - 7*q^178 - 3*q^180 - 5*q^181 + 4*q^182 + 7*q^185 - 6*q^187 + 6*q^188 + 6*q^190 + 7*q^194 - 15*q^197 - 6*q^198 + 12*q^200 - 3*q^202 - 18*q^203 - 9*q^205 + 6*q^206 + 1

In [494]:
x = ff1/ff2;x

q^-1 - q + 2*q^2 - q^4 - q^6 + 3*q^7 - q^8 - 2*q^9 + 2*q^10 - 2*q^11 + 6*q^12 - 4*q^13 - 6*q^14 + 9*q^15 - 6*q^16 + 9*q^17 - 6*q^18 - 15*q^19 + 29*q^20 - 19*q^21 + 5*q^22 + 4*q^23 - 35*q^24 + 70*q^25 - 47*q^26 - 17*q^27 + 54*q^28 - 85*q^29 + 135*q^30 - 96*q^31 - 74*q^32 + 212*q^33 - 232*q^34 + 227*q^35 - 125*q^36 - 241*q^37 + 625*q^38 - 616*q^39 + 309*q^40 + 78*q^41 - 755*q^42 + 1527*q^43 - 1428*q^44 + 247*q^45 + 1101*q^46 - 2280*q^47 + 3308*q^48 - 2757*q^49 - 481*q^50 + 4405*q^51 - 6485*q^52 + 6581*q^53 - 3842*q^54 - 3826*q^55 + 13277*q^56 - 16817*q^57 + 11933*q^58 - 1001*q^59 - 16063*q^60 + 34742*q^61 - 38809*q^62 + 18083*q^63 + 17527*q^64 - 54629*q^65 + 83073*q^66 - 77540*q^67 + 14240*q^68 + 83760*q^69 - 161844*q^70 + 184878*q^71 - 125578*q^72 - 45103*q^73 + 278522*q^74 - 427622*q^75 + 379001*q^76 - 122789*q^77 - 310001*q^78 + 794403*q^79 - 1014418*q^80 + 680961*q^81 + 158908*q^82 - 1203035*q^83 + 2060980*q^84 - 2147990*q^85 + 913922*q^86 + 1423504*q^87 - 3778983*q^88 + 4946015*q^89

In [495]:
q = x.parent().gen()

In [496]:
y = (x.derivative() / (ff2/q)); show(y.truncate(80))

-q^-3 + 3 - 9*q^2 + 3*q^3 + 15*q^5 - 5*q^6 - 42*q^7 + 48*q^8 - 12*q^9 + 21*q^10 - 9*q^11 - 153*q^12 + 267*q^13 - 99*q^14 - 91*q^15 + 156*q^16 - 516*q^17 + 945*q^18 - 420*q^19 - 783*q^20 + 1361*q^21 - 1683*q^22 + 2301*q^23 - 940*q^24 - 3348*q^25 + 6336*q^26 - 5558*q^27 + 3855*q^28 + 372*q^29 - 11872*q^30 + 22251*q^31 - 17457*q^32 + 1715*q^33 + 15405*q^34 - 40308*q^35 + 63822*q^36 - 45606*q^37 - 20799*q^38 + 87496*q^39 - 132444*q^40 + 154584*q^41 - 84420*q^42 - 125211*q^43 + 343878*q^44 - 409773*q^45 + 315426*q^46 - 31932*q^47 - 530305*q^48 + 1109652*q^49 - 1145754*q^50 + 487080*q^51 + 593358*q^52 - 1959468*q^53 + 3129887*q^54 - 2764884*q^55 + 168738*q^56 + 3468488*q^57 - 6558468*q^58 + 7919829*q^59 - 5293131*q^60 - 2984721*q^61 + 13577550*q^62 - 19903510*q^63 + 17956083*q^64 - 5694273*q^65 - 17872163*q^66 + 44305962*q^67 - 54336612*q^68 + 34815523*q^69 + 11501814*q^70 - 73798809*q^71 + 128719343*q^72 - 131264820*q^73 + 47233218*q^74 + 102670203*q^75 - 256163295*q^76 + 340914804*q^77 - 269626719*q^78 - 18158082*q^79

In [497]:
L = []
L.append([(y^2)[n] for n in range(-6,96)])
L.append([(y)[n] for n in range(-6,96)])
L.append([(x*y)[n] for n in range(-6,96)])
L.append([(x^2*y)[n] for n in range(-6,96)])
for i in range(7):
    t = x^i
    L.append([t[n] for n in range(-6,96)])

In [499]:
matrix(L).change_ring(CyclotomicField(12)).kernel()

Vector space of degree 11 and dimension 0 over Cyclotomic Field of order 12 and degree 4
Basis matrix:
[]

In [498]:
matrix(L).kernel().basis()

[

]

In [513]:
for p in prime_range(370):
    if p == 37:
        continue
    print(p,Ji.frobenius_polynomial(p).change_ring(a2.parent()).factor())

2 (x^2 - a2*x - 2*a2 - 2) * (x^2 + (a2 + 1)*x + 2*a2)
3 (x - a2 - 2) * (x + a2 - 1) * (x - a2 + 1) * (x + a2 + 2)
5 (x^2 - a2*x - 5*a2 - 5) * (x^2 + (a2 + 1)*x + 5*a2)
7 (x^2 - 2*a2*x - 7*a2 - 7) * (x^2 + (2*a2 + 2)*x + 7*a2)
11 (x^2 + 2*x + 11)^2
13 (x - a2 - 4) * (x + a2 - 3) * (x - 3*a2 + 1) * (x + 3*a2 + 4)
17 (x^2 - 3*a2*x - 17*a2 - 17) * (x^2 + (3*a2 + 3)*x + 17*a2)
19 (x^2 + (-6*a2 - 6)*x + 19*a2) * (x^2 + 6*a2*x - 19*a2 - 19)
23 (x^2 + 4*x + 23)^2
29 (x^2 - 9*x + 29)^2
31 (x^2 + 10*x + 31)^2
41 (x^2 + (-9*a2 - 9)*x + 41*a2) * (x^2 + 9*a2*x - 41*a2 - 41)
43 (x^2 - 2*x + 43)^2
47 (x^2 - 6*x + 47)^2
53 (x^2 + (-2*a2 - 2)*x + 53*a2) * (x^2 + 2*a2*x - 53*a2 - 53)
59 (x^2 + (-4*a2 - 4)*x + 59*a2) * (x^2 + 4*a2*x - 59*a2 - 59)
61 (x - 5*a2 - 9) * (x + 5*a2 - 4) * (x - 4*a2 + 5) * (x + 4*a2 + 9)
67 (x^2 + (-10*a2 - 10)*x + 67*a2) * (x^2 + 10*a2*x - 67*a2 - 67)
71 (x^2 - 6*a2*x - 71*a2 - 71) * (x^2 + (6*a2 + 6)*x + 71*a2)
73 (x - 8*a2 + 1)^2 * (x + 8*a2 + 9)^2
79 (x^2 - 10*a2*x - 79*a2 

In [505]:
Ji.lseries()(1)

0.419232609839550 - 1.66533453693773e-16*I

In [506]:
Ji.tamagawa_number()

TypeError: tamagawa_number() missing 1 required positional argument: 'p'

In [510]:
Ji.torsion_subgroup(2)

Finite subgroup with invariants [2, 2, 2, 2] over QQ of Simple abelian subvariety 37cG1(1,37) of dimension 2 of J1(37)